In [1]:
import pandas as pd
import os
import numpy as np
from nltk.corpus import stopwords
from unidecode import unidecode
import tensorflow as tf
#import keras
from setuptools import distutils
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import spacy
import re

In [2]:
#read the path
file_path = "C:/Users/ASUS/Desktop/stage_pfe/data/data_excel"
#list all the files from the directory
file_list = os.listdir(file_path)
file_list


['accessoires 2023.xlsx',
 'hybride 2023.xlsx',
 'modèle marque 2023.xlsx',
 'neuve2023.xlsx',
 'occasion 2023.xlsx',
 'type voiture 2023.xlsx']

In [3]:
def data_merge(file_path,file_list,header):
    dfs = []
    for file_name in file_list:
        file_full_path = os.path.join(file_path, file_name)
        df_temp = pd.read_excel(file_full_path, header= header)
        dfs.append(df_temp)
    df = pd.concat(dfs, ignore_index=True)    
    return df

In [4]:
dataset = data_merge(file_path,file_list,2)

In [5]:
dataset

,Keyword,Currency,Avg. monthly searches,Variation sur trois mois,Variation YoY,Competition,Competition (indexed value),Top of page bid (low range),Top of page bid (high range),Ad impression share,...,In plan?,Searches: Jan 2023,Searches: Feb 2023,Searches: Mar 2023,Searches: Apr 2023,Searches: May 2023,Searches: Jun 2023,Searches: Jul 2023,Searches: Aug 2023,Searches: Sep 2023
0,accessoires auto tunisie,USD,260.0,-0.19,0.24,Faible,2.0,NaN,NaN,NaN,...,NaN,320.0,170.0,260.0,210.0,260.0,210.0,320.0,260.0,260.0
1,accessoire voiture tunisie,USD,880.0,0.22,0,Faible,2.0,0.01,0.03,NaN,...,NaN,880.0,720.0,880.0,880.0,880.0,720.0,720.0,880.0,880.0
2,maison volkswagen tunisie pièce de rechange,USD,140.0,-0.18,0.27,Faible,0.0,NaN,NaN,NaN,...,NaN,110.0,90.0,110.0,90.0,140.0,140.0,170.0,170.0,140.0
3,enjoliveur tunisie,USD,110.0,0,0.22,Faible,3.0,NaN,NaN,NaN,...,NaN,110.0,110.0,110.0,90.0,90.0,110.0,110.0,140.0,110.0
4,accessoire voiture tunisie prix,USD,90.0,0.29,0.29,Faible,1.0,NaN,NaN,NaN,...,NaN,90.0,70.0,90.0,110.0,90.0,70.0,70.0,50.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5751,location utilitaire tunis,USD,10.0,∞,0,Faible,33.0,NaN,NaN,NaN,...,NaN,0.0,10.0,10.0,10.0,10.0,10.0,0.0,10.0,10.0
5752,circuit 4x4 tunisie marmara,USD,0.0,0,0,Inconnu,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5753,excursion 4x4 hammamet,USD,10.0,0,∞,Moyen,37.0,0.04,0.33,NaN,...,NaN,0.0,0.0,0.0,0.0,10.0,10.0,10.0,10.0,10.0
5754,raid 4x4 en tunisie,USD,10.0,∞,0,Faible,0.0,NaN,NaN,NaN,...,NaN,10.0,0.0,10.0,10.0,0.0,10.0,0.0,0.0,10.0


In [6]:
def show_dataset(data):
    return data

**Data understanding**

In [7]:
dataset.shape

(5756, 23)

In [8]:
dataset.columns

Index(['Keyword', 'Currency', 'Avg. monthly searches',
       'Variation sur trois mois', 'Variation YoY', 'Competition',
       'Competition (indexed value)', 'Top of page bid (low range)',
       'Top of page bid (high range)', 'Ad impression share',
       'Organic impression share', 'Organic average position', 'In account?',
       'In plan?', 'Searches: Jan 2023', 'Searches: Feb 2023',
       'Searches: Mar 2023', 'Searches: Apr 2023', 'Searches: May 2023',
       'Searches: Jun 2023', 'Searches: Jul 2023', 'Searches: Aug 2023',
       'Searches: Sep 2023'],
      dtype='object')

In [9]:
data_types = dataset.dtypes
data_types

Keyword                          object
Currency                         object
Avg. monthly searches           float64
Variation sur trois mois         object
Variation YoY                    object
Competition                      object
Competition (indexed value)     float64
Top of page bid (low range)     float64
Top of page bid (high range)    float64
Ad impression share             float64
Organic impression share        float64
Organic average position        float64
In account?                     float64
In plan?                        float64
Searches: Jan 2023              float64
Searches: Feb 2023              float64
Searches: Mar 2023              float64
Searches: Apr 2023              float64
Searches: May 2023              float64
Searches: Jun 2023              float64
Searches: Jul 2023              float64
Searches: Aug 2023              float64
Searches: Sep 2023              float64
dtype: object

In [10]:
missing_values = dataset.isnull().sum()
missing_values

Keyword                            0
Currency                           0
Avg. monthly searches           1244
Variation sur trois mois           0
Variation YoY                      0
Competition                        0
Competition (indexed value)     3143
Top of page bid (low range)     5681
Top of page bid (high range)    5681
Ad impression share             5756
Organic impression share        5756
Organic average position        5756
In account?                     5756
In plan?                        5756
Searches: Jan 2023              1244
Searches: Feb 2023              1244
Searches: Mar 2023              1244
Searches: Apr 2023              1244
Searches: May 2023              1244
Searches: Jun 2023              1244
Searches: Jul 2023              1244
Searches: Aug 2023              1244
Searches: Sep 2023              1244
dtype: int64

In [11]:
dataset.describe()

,Avg. monthly searches,Competition (indexed value),Top of page bid (low range),Top of page bid (high range),Ad impression share,Organic impression share,Organic average position,In account?,In plan?,Searches: Jan 2023,Searches: Feb 2023,Searches: Mar 2023,Searches: Apr 2023,Searches: May 2023,Searches: Jun 2023,Searches: Jul 2023,Searches: Aug 2023,Searches: Sep 2023
count,4512.000000,2613.000000,75.000000,75.000000,0.0,0.0,0.0,0.0,0.0,4512.000000,4512.000000,4512.000000,4512.000000,4512.000000,4512.000000,4512.000000,4512.000000,4512.000000
mean,221.560284,1.431688,0.207200,0.827200,NaN,NaN,NaN,NaN,NaN,218.047429,185.948582,218.282358,217.726064,224.388298,191.054965,221.826241,247.841312,228.956117
std,1552.874011,6.641586,0.497468,0.812069,NaN,NaN,NaN,NaN,NaN,1551.238641,1270.293199,1552.575623,1549.245973,1559.983808,1350.866991,1556.401100,1723.161309,1576.764559
min,0.000000,0.000000,0.010000,0.030000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,0.000000,0.040000,0.195000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.000000,0.000000,0.090000,0.580000,NaN,NaN,NaN,NaN,NaN,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
75%,40.000000,0.000000,0.270000,1.570000,NaN,NaN,NaN,NaN,NaN,40.000000,40.000000,40.000000,40.000000,50.000000,40.000000,40.000000,50.000000,40.000000
max,22200.000000,71.000000,4.300000,5.170000,NaN,NaN,NaN,NaN,NaN,22200.000000,18100.000000,22200.000000,22200.000000,22200.000000,18100.000000,22200.000000,22200.000000,22200.000000


In [12]:
dataset.describe(include=['object'])

,Keyword,Currency,Variation sur trois mois,Variation YoY,Competition
count,5756,5756,5756,5756,5756
unique,5465,1,86,136,4
top,prix rav4 hybride tunisie,USD,0,0,Inconnu
freq,3,5756,2704,2398,3143


***Data cleaning***

**Missing values**

In [13]:
def drop_empty_columns(df):
    data = pd.DataFrame()
    for col_name in df.columns:
        #print(col_name)
        if (df[col_name].isnull().sum() == len(df)):
            data = df.drop(col_name, axis=1)
            print(col_name)
    print("\n ==> These columns were deleted succesfully")        
    return data 
  

In [14]:
cleaned_df = drop_empty_columns(dataset)

Ad impression share
Organic impression share
Organic average position
In account?
In plan?

 ==> These columns were deleted succesfully


In [15]:
cleaned_df.columns

Index(['Keyword', 'Currency', 'Avg. monthly searches',
       'Variation sur trois mois', 'Variation YoY', 'Competition',
       'Competition (indexed value)', 'Top of page bid (low range)',
       'Top of page bid (high range)', 'Ad impression share',
       'Organic impression share', 'Organic average position', 'In account?',
       'Searches: Jan 2023', 'Searches: Feb 2023', 'Searches: Mar 2023',
       'Searches: Apr 2023', 'Searches: May 2023', 'Searches: Jun 2023',
       'Searches: Jul 2023', 'Searches: Aug 2023', 'Searches: Sep 2023'],
      dtype='object')

**duplicates**

In [16]:
len(cleaned_df["Keyword"].unique())

5465

In [17]:
len(cleaned_df["Keyword"])

5756

In [18]:
cleaned_df.duplicated(subset= ['Keyword']).sum()

291

In [19]:
cleaned_df = cleaned_df.drop_duplicates(subset=['Keyword'], keep='first').reset_index(drop=True)


***Data preparation + PRETRAITEMENT***

In [20]:
dataset = pd.DataFrame()
dataset.loc[:, "keyword"] = cleaned_df.iloc[:,0]


In [21]:
dataset

,keyword
0,accessoires auto tunisie
1,accessoire voiture tunisie
2,maison volkswagen tunisie pièce de rechange
3,enjoliveur tunisie
4,accessoire voiture tunisie prix
...,...
5460,location utilitaire tunis
5461,circuit 4x4 tunisie marmara
5462,excursion 4x4 hammamet
5463,raid 4x4 en tunisie


give the app user the chance to concatenate two words with specific concatenater like - , _ , '', ..

i will use this function do concatenate d + max to detect isuzu dmax correctly if found 

In [22]:
def concatenate_words_in_sentence(sentence, word1, word2, concat_str):
    words = sentence.split()
    
    # Find the positions of the two specific words
    try:
        index1 = words.index(word1)
        index2 = words.index(word2)
    except ValueError:
        # If either word is not found, return the original sentence
        return sentence
    
    # Ensure the words are in the correct order
    if index1 > index2:
        index1, index2 = index2, index1
    
    # Concatenate the words with the given string
    concatenated = word1 + concat_str + word2
    
    # Replace the two words in the sentence with the concatenated string
    new_words = words[:index1] + [concatenated] + words[index1+1:index2] + words[index2+1:]
    
    # Join the words back into a sentence
    new_sentence = ' '.join(new_words)
    
    return new_sentence


In [23]:
word1 = "d"
word2 = "max"
concat_str = ""
dataset["keyword"] = dataset["keyword"].apply(lambda x: concatenate_words_in_sentence (x, word1, word2, concat_str))

correcting the wrong spelling of questions so that they don't be labeled with model if they contain a letter that refers to a car model . for example:  "citroen c zero". 

most of the users don't write properly the guestion structure: 

for instance:
qu'est ce que c'est is written: =>   qu est ce que c est. 

as a result, after tokenization, stopword like c'est won't be removed when performing stopwords deletion.

In [24]:
# Load the French model
nlp = spacy.load("fr_core_news_sm")

# Define common question words and patterns
question_words = {
    "qui", "que", "quoi", "où", "quand", "pourquoi", "comment", "combien",
    "qu'", "est-ce", "ça"
}

def is_question(sentence):
    doc = nlp(sentence)
    for token in doc:
        # Check if the token is a question word or if the sentence ends with a question mark
        if token.text.lower() in question_words or (token.is_punct and token.text == '?'):
            return True
    return False

def preprocess_query(query):
    if is_question(query):
        # Handle common question patterns that involve "c"
        query = query.replace("qu est ce que c est", "qu'est-ce que c'est")
        query = query.replace("qu'est ce que c'est", "qu'est-ce que c'est")
        query = query.replace("qu est-ce que c est", "qu'est-ce que c'est")
        query = query.replace("qu est ce c'est", "qu'est-ce que c'est")
        query = query.replace("comment ca marche", "comment ça marche")
        query = query.replace("comment ca fonctionne", "comment ça fonctionne")
        query = query.replace("c est", "c'est")
        query = query.replace("c etait", "c'était")
        

        # Handle "c" in questions more generally by checking for patterns
        question_patterns = [" qu est ce que ", " qu'est-ce que ", " c'est ", " est-ce "]
        for pattern in question_patterns:
            query = query.replace(pattern + "c ", pattern + "c' ")
    return query


# Apply preprocessing to each query
preprocessed_queries = [preprocess_query(query) for query in dataset["keyword"]]

# for original, preprocessed in zip(list(dataset["keyword"]), preprocessed_queries):
#     print(f"Original: {original}")
#     print(f"Preprocessed: {preprocessed}")
#     print("-" * 40)

# Apply preprocessing to the DataFrame
dataset["Preprocessed_Keyword"] = ''
dataset["Preprocessed_Keyword"] = dataset["keyword"].apply(preprocess_query)

dataset


,keyword,Preprocessed_Keyword
0,accessoires auto tunisie,accessoires auto tunisie
1,accessoire voiture tunisie,accessoire voiture tunisie
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie pièce de rechange
3,enjoliveur tunisie,enjoliveur tunisie
4,accessoire voiture tunisie prix,accessoire voiture tunisie prix
...,...,...
5460,location utilitaire tunis,location utilitaire tunis
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara
5462,excursion 4x4 hammamet,excursion 4x4 hammamet
5463,raid 4x4 en tunisie,raid 4x4 en tunisie


fixing the l' issue : it is mistakenly detected as model

In [25]:
# french_vowels = ['a', 'e', 'i', 'o', 'u', 'y','à', 'â', 'ä', 'é', 'è', 'ê', 'ë', 'î', 'ï', 'ô', 'ö', 'ù', 'û', 'ü', 'ÿ','A', 'E', 'I', 'O', 'U', 'Y','À', 'Â', 'Ä', 'É', 'È', 'Ê', 'Ë', 'Î', 'Ï', 'Ô', 'Ö', 'Ù', 'Û', 'Ü', 'Ÿ']

# for row in range(len(dataset)):
#     words = dataset["keyword"][row].split()
#     for idx, word in enumerate(words):
#         if word == "l":
#             word_index = idx
#             if word_index < len(words) - 1:
#                 nxt_word = words[word_index + 1]
#                 if nxt_word[0] in french_vowels:
#                     dataset.at[row, "Preprocessed_Keyword"] = concatenate_words_in_sentence(dataset["keyword"][row], word, nxt_word, "'")
                    


deleting digit + places / place pattern :
this pretraitement is used to correct wrong labeling issue:
eg: "voiture 10 places tunisie" is labeled with "modele" because function miss took it as "aro 10"  

In [26]:
def remove_digit_places(text):
    # Regular expression pattern to match digits followed by "places" or "place"
    pattern = r'\b\d+\s*(?:places|place)\b'
    # Replace matched pattern with an empty string
    return re.sub(pattern, '', text)

# Example usage:
#text = "voiture 10 places tunisie"
#cleaned_text = remove_digit_places(text)
#print(cleaned_text)

In [27]:
dataset.loc[:, 'Preprocessed_Keyword'] = dataset.loc[:, 'Preprocessed_Keyword'].apply(remove_digit_places)

handling the false labeling of digit + kw for electric car related search queries. when the digit coincides with a car model then the search query will be labeled with model when the digit did not refer to a model in the first place  + (digit + km pattern)

In [28]:
def remove_digit_kw_km(text):
    # Regular expression to match the searched pattern 
    pattern = r'\b\d+\s*(?:kwh|kw|k.w.|kw-h|kwhr|kilowatt-hour|kilowatt-heure|km|k.m.|kilomètre|kilometre|kilometer)\b'
    # Replace matched pattern with an empty string
    return re.sub(pattern, '', text)

In [29]:
dataset.loc[:, 'Preprocessed_Keyword'] = dataset.loc[:, 'Preprocessed_Keyword'].apply(remove_digit_kw_km)

**replacing accented "e" with unaccented "e"**

In [30]:
#to resolve an issue : different ways to write certain brand and model names 
accents = {
    'é': 'e',
    'è': 'e',
    'ê': 'e',
    'ë': 'e',
}

for accented_char, unaccented_char in accents.items():
    dataset["Preprocessed_Keyword"] = dataset["Preprocessed_Keyword"].str.replace(accented_char, unaccented_char, regex=False)

**tokenization**

In [31]:
# Applying text_to_word_sequence function to each keyword and store the tokens in the Tokenized_Keyword column
#dataset['Tokenized_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: text_to_word_sequence(x))

In [32]:
from nltk.tokenize import word_tokenize
dataset['Preprocessed_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: x.lower())
dataset['Tokenized_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: word_tokenize(x))

changing "auto" to "voiture" when "auto" exists by itself in the search query. 
this preprocessing step is crucial to avoid labeling generic search queries with model/brand if they contain auto as substring (eg: auto union)

In [33]:
def replace_auto_with_voiture(words):
    if "auto" in words and "union" not in words:
        words = ["voiture" if word == "auto" else word for word in words]
    return words

# Apply the function to each cell in the "Tokenized_Keyword" column
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(replace_auto_with_voiture)

print(dataset)


                                          keyword  \
0                        accessoires auto tunisie   
1                      accessoire voiture tunisie   
2     maison volkswagen tunisie pièce de rechange   
3                              enjoliveur tunisie   
4                 accessoire voiture tunisie prix   
...                                           ...   
5460                    location utilitaire tunis   
5461                  circuit 4x4 tunisie marmara   
5462                       excursion 4x4 hammamet   
5463                          raid 4x4 en tunisie   
5464                           voyage tunisie 4x4   

                             Preprocessed_Keyword  \
0                        accessoires auto tunisie   
1                      accessoire voiture tunisie   
2     maison volkswagen tunisie piece de rechange   
3                              enjoliveur tunisie   
4                 accessoire voiture tunisie prix   
...                                          

**deleting stopwords**

In [34]:
fr_stops = set(stopwords.words('french')) 

In [35]:
fr_stops

{'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'd',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'il',
 'ils',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes',
 'moi',
 'mon',
 'même',
 'n',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ont',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 's',
 'sa',
 'se',
 'sera',
 'serai',
 'seraient',
 'serais',
 'serait',


In [36]:
def remove_stopwords(tokens,stopword):
    return [token for token in tokens if token.lower() not in stopword]

**feature enginnering + labeling**

**detection prix**

In [37]:
def detect_prix(tokenized_keyword_list):
    res = False
    for i in range(len(tokenized_keyword_list)):
        if tokenized_keyword_list[i] == "prix":
            res = True
    return res

            

In [38]:
def prix_label(dataset):
    dataset["prix"] = ''
    for i in range(len(dataset)):
        tokenized_keyword_list = dataset["Tokenized_Keyword"].iloc[i]
        if detect_prix(tokenized_keyword_list) == True:
            dataset.at[i, "prix"] = "prix"
        else:
            dataset.at[i, "prix"] = "not mentioned"
    return dataset
    

In [39]:
prix_label(dataset)

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, voiture, tunisie]",not mentioned
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, de, recha...",not mentioned
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned
4,accessoire voiture tunisie prix,accessoire voiture tunisie prix,"[accessoire, voiture, tunisie, prix]",prix
...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, en, tunisie]",not mentioned


In [40]:
dataset["prix"].value_counts()

prix
not mentioned    2883
prix             2582
Name: count, dtype: int64

In [41]:
def detect_prix_pattern(keyword):
    currency_symbols = r'[$€£¥₹₽₩₪]'
    currency_names = r'\b(euros?|dollars?|dinars?|pounds?|livres?|yen|rupees?|roubles?|rubles?|won|shekels?)\b'
    
    # Regular expression pattern to match "prix" or a digit followed by a currency name/symbol
    pattern = re.compile(r'prix|\b\d+\s*(%s|%s)' % (currency_symbols, currency_names), re.IGNORECASE)
    
    return bool(pattern.search(keyword))

In [42]:
for index, label in dataset["prix"].items():
    if label == "not mentioned":
        if detect_prix_pattern(dataset["Preprocessed_Keyword"].iloc[index]):
            dataset.at[index, "prix"] = "prix"
        else:
            dataset.at[index, "prix"] = "not mentioned"



In [43]:
dataset["prix"].value_counts()

prix
not mentioned    2844
prix             2621
Name: count, dtype: int64

deleting the price pattern to avoid labeling the price with model when it coincides with an actual car model 

In [44]:
def delete_prix_pattern(keyword):
    currency_symbols = r'[$€£¥₹₽₩₪]'
    currency_names = r'\b(euros?|dollars?|dinars?|pounds?|livres?|yen|rupees?|roubles?|rubles?|won|shekels?)\b'
    
    # Regular expression pattern to match "prix" or a digit followed by a currency name/symbol
    pattern = re.compile(r'prix|\b\d+\s*(%s|%s)' % (currency_symbols, currency_names), re.IGNORECASE)
    
    # Replace matched patterns with an empty string
    return pattern.sub('', keyword).strip()

In [45]:
dataset['Preprocessed_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: delete_prix_pattern(x))

**detection hybride / éléctrique (hybrid/electric)**

In [46]:
# def detect_hybrid_electric(tokenized_keyword_list):
#     res = False
#     for i in range(len(tokenized_keyword_list)):
#         if ((tokenized_keyword_list[i] == 'hybride') or (tokenized_keyword_list[i] == 'electrique')):
#             res = True  
            
#     return res


In [47]:
def detect_hybrid_electric(tokenized_keyword_list):
    res = False
    for i in range(len(tokenized_keyword_list)):
        if tokenized_keyword_list[i] in ['hybride', 'electrique','electric','hybrid',"électrique","éléctrique","electriques","hybrides","hybrids"]:
            res = True  
            
    return res

In [48]:
def hybrid_electric_label(dataset):
    dataset["hybride/éléctrique"] = ''
    for i in range(len(dataset)):
        tokenized_keyword_list = dataset["Tokenized_Keyword"].iloc[i]
        if detect_hybrid_electric(tokenized_keyword_list) == True:
            dataset.at[i, "hybride/éléctrique"] = "hybride/éléctrique"
        else:
            dataset.at[i, "hybride/éléctrique"] = "not mentioned"
        
    return dataset

In [49]:
hybrid_electric_label(dataset)

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, voiture, tunisie]",not mentioned,not mentioned
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned,not mentioned
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, de, recha...",not mentioned,not mentioned
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned,not mentioned
4,accessoire voiture tunisie prix,accessoire voiture tunisie,"[accessoire, voiture, tunisie, prix]",prix,not mentioned
...,...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned,not mentioned
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned,not mentioned
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned,not mentioned
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, en, tunisie]",not mentioned,not mentioned


In [50]:
dataset["hybride/éléctrique"].value_counts()

hybride/éléctrique
not mentioned         3503
hybride/éléctrique    1962
Name: count, dtype: int64

deleting hybride/éléctrique from keywords to avoid confusion 

In [51]:
def remove_hybrid_electric(tokens):
    cleaned_tokens = [token for token in tokens if token not in ['hybride', 'electrique','electric','hybrid',"électrique","éléctrique","electriques","hybrides","hybrids"]]
    return ' '.join(cleaned_tokens)

In [52]:
dataset["Preprocessed_Keyword"] = dataset["Preprocessed_Keyword"].apply(lambda x: remove_hybrid_electric(x.split()))

**detection occasion**

In [53]:
def detect_occasion(tokenized_keyword_list):
    res = False
    for i in range(len(tokenized_keyword_list)):
        if (tokenized_keyword_list[i] == 'occasion'):
            res = True
            
    return res

In [54]:
def occasion_label(dataset):
    dataset["occasion"] = ''
    for i in range(len(dataset)):
        tokenized_keyword_list = dataset["Tokenized_Keyword"].iloc[i]
        if detect_occasion(tokenized_keyword_list) == True:
            dataset.at[i, "occasion"] = "occasion"
        else:
            dataset.at[i, "occasion"] = "not mentioned"
        
    return dataset

In [55]:
occasion_label(dataset)

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, voiture, tunisie]",not mentioned,not mentioned,not mentioned
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned,not mentioned,not mentioned
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, de, recha...",not mentioned,not mentioned,not mentioned
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned,not mentioned,not mentioned
4,accessoire voiture tunisie prix,accessoire voiture tunisie,"[accessoire, voiture, tunisie, prix]",prix,not mentioned,not mentioned
...,...,...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned,not mentioned,not mentioned
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned,not mentioned,not mentioned
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned,not mentioned,not mentioned
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, en, tunisie]",not mentioned,not mentioned,not mentioned


In [56]:
dataset["occasion"].value_counts()

occasion
not mentioned    4473
occasion          992
Name: count, dtype: int64

In [57]:
def remove_occasion(tokens):
    cleaned_tokens = [token for token in tokens if token != "occasion"]
    return ' '.join(cleaned_tokens)

In [58]:
dataset["Preprocessed_Keyword"] = dataset["Preprocessed_Keyword"].apply(lambda x: remove_occasion(x.split()))

In [59]:
def detect_neuve(tokenized_keyword_list):
    res = False
    for i in range(len(tokenized_keyword_list)):
        if (tokenized_keyword_list[i] == 'neuve') or (tokenized_keyword_list[i] == 'neuf') or (tokenized_keyword_list[i] == 'neufs') or (tokenized_keyword_list[i] == 'neuves')  :
            res = True
            
    return res

In [60]:
def neuve_label(dataset):
    dataset["neuve"] = ''
    for i in range(len(dataset)):
        tokenized_keyword_list = dataset["Tokenized_Keyword"].iloc[i]
        if detect_neuve(tokenized_keyword_list) == True:
            dataset.at[i, "neuve"] = "neuve"
        else:
            dataset.at[i, "neuve"] = "not mentioned"
        
    return dataset

In [61]:
neuve_label(dataset)

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,neuve
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, voiture, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, de, recha...",not mentioned,not mentioned,not mentioned,not mentioned
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
4,accessoire voiture tunisie prix,accessoire voiture tunisie,"[accessoire, voiture, tunisie, prix]",prix,not mentioned,not mentioned,not mentioned
...,...,...,...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned,not mentioned,not mentioned,not mentioned
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned,not mentioned,not mentioned,not mentioned
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned,not mentioned,not mentioned,not mentioned
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, en, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned


In [62]:
dataset["neuve"].value_counts()

neuve
not mentioned    5180
neuve             285
Name: count, dtype: int64

deleting car types to avoid false labeling as "model"

In [63]:
type_voitures = [
    "Berline", "Sedan",
    "Coupé", "Coupe",
    "Cabriolet", "Convertible",
    "Break", "Station Wagon", "Estate",
    "Monospace", "Minivan",
    "Tout-terrain", "SUV", "Sport Utility","utilitaire",
    "Camionnette", "Pickup Truck", "Pickup","pick-up","pick up",
    "Sport", "Sports", "Sports Car",
    "Luxe", "Luxury",
    "Électrique", "Electric","éléctrique","electrique",
    "Hybride", "Hybrid",
    "Fourgonnette", "Van",
    "Utilitaire", "Utility",
    "Camion", "Truck",
    "Compacte", "Compact",
    "Sous-compacte", "Subcompact",
    "Familiale", "Family",
    "Course", "Race",
    "Rallye", "Rally",
    "Collection", "Classic",
    "Économique", "Economy",
    "Limousine",
    "Roadster",
    "Supercar",
    "Muscle", "Muscle",
    "Compact", "Compact",
    "Crossover", "Crossover",
    "Tout-terrain", "Off-road",
    "Micro", "Micro",
    "Kit",
    "Dune Buggy",
    "Targa",
    "Shooting Brake",
    "Grand Tourer", "GT",
    "Électrique", "Electric",
    "Kei",
    "Concept",
    "Amphibie", "Amphibious",
    "Blindée", "Armored",
    "Corbillard", "Hearse", "funeral",
    "Hatchback",
    "Automobile",
    "MPV",
    "City Car","cross","sport","berlinette","crossback","4 * 4","4*4","4x4"
]

In [64]:
type_voitures_lower = [car_type.lower() for car_type in type_voitures]

In [65]:
# def delete_car_type(keywords_list, type_voitures_lower_list):
#     filtered_keywords = []
#     for keyword in keywords_list:
#         found = False
#         for car_type in type_voitures_lower_list:
#             if keyword == car_type:
#                 found = True
#                 break
#         if not found:
#             filtered_keywords.append(keyword)
#     return filtered_keywords



In [66]:
def delete_car_type(keywords_list, type_voitures_lower_list):
    return [keyword for keyword in keywords_list if keyword not in type_voitures_lower_list]

In [67]:
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(lambda x: delete_car_type(x, type_voitures_lower))

deleting digit + voiture pattern to avoid false labeling as model (detecting the digit as a model)

exp query: les 10 voitures electriques les moins cheres

In [68]:
def remove_digit_voitures(text):
    # Regular expression pattern to match digits followed by "places" or "place"
    pattern = r'\b\d+\s*(?:voitures|voiture)\b'
    # Replace matched pattern with an empty string
    return re.sub(pattern, '', text)

In [69]:
dataset.loc[:, 'Preprocessed_Keyword'] = dataset.loc[:, 'Preprocessed_Keyword'].apply(remove_digit_voitures)

***tokenization***

In [70]:
# re-applying text_to_word_sequence function to each keyword and storing the tokens in the Tokenized_Keyword column after finishing all the preprocessing tasks
#dataset['Tokenized_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: text_to_word_sequence(x))

In [71]:
from nltk.tokenize import word_tokenize
dataset['Preprocessed_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: x.lower())
dataset['Tokenized_Keyword'] = dataset['Preprocessed_Keyword'].apply(lambda x: word_tokenize(x))

**data collection and preparation**

collecting data about different car brands and their respective models

1) scraped data

In [72]:
scraped_df = pd.read_csv(r"C:/Users/ASUS/Desktop/stage_pfe/scraped_data.csv")
scraped_df

,brand,model,models_without_brand
0,Abarth,Abarth 500,500
1,Abarth,Abarth 600e,600e
2,Abarth,Abarth 124,124
3,Abarth,Abarth Grande Punto,Grande Punto
4,Abarth,Abarth Punto Evo,Punto Evo
...,...,...,...
3327,Xinyang,Xinyang Electric Utility,Electric Utility
3328,Zagato,Zagato Milano,Milano
3329,Zastava,Zastava 1100,1100
3330,Zastava,Zastava 1300,1300


In [73]:
#FUNCTION THAT LOWERCASES THE DF
def lowercase_df(df):
    df_lower = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    return df_lower


In [74]:
scraped_data = lowercase_df(scraped_df)
scraped_data

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2884285594.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lower = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,brand,model,models_without_brand
0,abarth,abarth 500,500
1,abarth,abarth 600e,600e
2,abarth,abarth 124,124
3,abarth,abarth grande punto,grande punto
4,abarth,abarth punto evo,punto evo
...,...,...,...
3327,xinyang,xinyang electric utility,electric utility
3328,zagato,zagato milano,milano
3329,zastava,zastava 1100,1100
3330,zastava,zastava 1300,1300


correcting the model_without_brand column

In [75]:
for i in range(len(scraped_data)):
    brand = scraped_data["brand"][i]
    model = scraped_data["model"][i]
    # Check if the model starts with the brand name
    if model.startswith(brand):
        model = model[len(brand):].strip()
    scraped_data.loc[i, "models_without_brand"] = model



In [76]:
path = "corr_scraped_data.csv"
scraped_data.to_csv(path)

In [77]:
scraped_data

,brand,model,models_without_brand
0,abarth,abarth 500,500
1,abarth,abarth 600e,600e
2,abarth,abarth 124,124
3,abarth,abarth grande punto,grande punto
4,abarth,abarth punto evo,punto evo
...,...,...,...
3327,xinyang,xinyang electric utility,electric utility
3328,zagato,zagato milano,milano
3329,zastava,zastava 1100,1100
3330,zastava,zastava 1300,1300


In [78]:
len(scraped_data)

3332

In [79]:
len(scraped_data["brand"].unique())

308

2) kaggle dataset

In [80]:
labelDf = pd.read_excel(r"C:/Users/ASUS/Desktop/stage_pfe/projet SEO/CarDB.xlsx", sheet_name = "CarModels")
labelDf

,ID,BRAND,MODEL,YEAR_,FUEL,SHIFTTYPE,MOTORVOLUME,CASETYPE,VARIANT
0,1,Audi,100,1993,Gasoline/LPG,Automatic transmission,1301-1600 cc,Sedan,2.0
1,2,Audi,100,2006,Gasoline/LPG,Automatic transmission,2001-2500 cc,Sedan,2.0
2,3,Audi,80,1982,Diesel,Manual Gear,-,Sedan,Audi 80 Series 1.6 D
3,4,Audi,80,1991,Diesel,Manual Gear,1301-1600 cc,Sedan,Audi 80 Series 1.6 D
4,5,Audi,A1,2011,Gasoline,Semi-Automatic Gear,1301-1600 cc,Hatchback 3 Doors,Audi A1 1.4 TFSI Ambition
...,...,...,...,...,...,...,...,...,...
6918,6919,Volvo,XC90,2015,Diesel,Semi-Automatic Gear,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription
6919,6920,Volvo,XC90,2017,Diesel,Automatic transmission,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription
6920,6921,Volvo,XC90,2018,Diesel,Automatic transmission,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription
6921,6922,Volvo,XC90,2019,Diesel,Automatic transmission,1801-2000 cc,Off Road,Volvo XC90 2.0 D5 Inscription


In [81]:
model_Brand_Df = pd.DataFrame()
model_Brand_Df["brand"] = labelDf["BRAND"]
model_Brand_Df["model"] = labelDf["BRAND"]+ " "+ labelDf["MODEL"]

In [82]:
model_Brand_Df

,brand,model
0,Audi,Audi 100
1,Audi,Audi 100
2,Audi,Audi 80
3,Audi,Audi 80
4,Audi,Audi A1
...,...,...
6918,Volvo,Volvo XC90
6919,Volvo,Volvo XC90
6920,Volvo,Volvo XC90
6921,Volvo,Volvo XC90


lowercase

In [83]:
model_Brand_Df = lowercase_df(model_Brand_Df)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2884285594.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lower = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [84]:
for i in range(len(model_Brand_Df)):
    brand = model_Brand_Df["brand"][i]
    model = model_Brand_Df["model"][i]
    # Check if the model starts with the brand name
    if model.startswith(brand):
        model = model[len(brand):].strip()
    model_Brand_Df.loc[i, "models_without_brand"] = model

In [85]:
model_Brand_Df

,brand,model,models_without_brand
0,audi,audi 100,100
1,audi,audi 100,100
2,audi,audi 80,80
3,audi,audi 80,80
4,audi,audi a1,a1
...,...,...,...
6918,volvo,volvo xc90,xc90
6919,volvo,volvo xc90,xc90
6920,volvo,volvo xc90,xc90
6921,volvo,volvo xc90,xc90


In [86]:
#detecting duplicates
model_Brand_Df.duplicated(subset= ['model']).sum()

6550

deleting duplicates 

In [87]:
if (model_Brand_Df.duplicated(subset= ['model']).sum() != 0):
    model_Brand_Df.drop_duplicates(subset= ['model'],keep= 'first', inplace= True)

In [88]:
print(model_Brand_Df["brand"].unique())

['audi' 'bmw' 'chevrolet' 'citroen' 'dacia' 'fiat' 'ford' 'honda'
 'hyundai' 'jeep' 'kia' 'mazda' 'mercedes' 'nissan' 'peugeot' 'porsche'
 'renault' 'seat' 'skoda' 'suzuki' 'tofaş' 'toyota' 'volkswagen' 'volvo']


In [89]:
model_Brand_Df["models_without_brand"].isna().sum()

0

3) manually collected data

this data was added manually due to missing brands and models for certain brands in the scraped and kaggle data and mainly for better labeling result

In [90]:
brands_models_add = {
    "toyota": ["prado", "yaris", "corolla", "rav 4", "hilux","agya","raize","rush"], 
    "geely": ["gx3", "coolray", "azkarra", "monjaro", "starray", "tugella", "okavango", "emgrand", "geometry c"],
    "kia": ["seltos","sonet","telluride"],
    "haval": ["h6", "h9", "jolion"],
    "jetour": ["x70", "t2", "dashing", "x70", "x90", "x95"],
    "micro": ["microlino"],
    "twike": ["3", "5"],
    "chery": ["qq", "tiggo", "tiggo 8", "tiggo 7", "tiggo 4", "tiggo 3x", "tiggo 2", "tiggo 1x"],
    "lormauto": ["twingo"],
    "ford": ["raptor"],
    "fiat": ["500 e"],
    "bmw" : ["520 i","316 i","e 36","m4","m5","e 46","e 92","320 i","318"],
    "great wall": ["wingle 5","poer bvm","poer bva"],
    "xev" : ["yoyo"],
    "biac" : ["eu 5","all new x7","x 55","x 35","u5 plus","bj80","bj60","bj40 plus","bj40 se","f 40"],
    "mitsubishi" : ["colt","asx","eclipse cross","space star"],
    "elaris" : ["dyo","lenn","finn","beo","jaco","caro","pio"],
    "ds" : ["3","4","7","9"],
    "togg" : ["t10x"],
    "renault" : ["4","4l","clio 5 societe","clio 5","clio 4 Rs","clio 4 estate","clio 3","clio 4","clio 3 ","clio 3 Rs","clio 3 estate","clio 2","clio 2 rs","clio 2 societe","clio williams","clio societe","megane 5","megane 2","megane 4","megane 3","megane 2","megane e","megane cabriolet","megane societe","megane coupe"],
    "volkswagen" : ["teramont","polo 5","polo 6","polo 7","polo 8","polo 6 entreprise","polo 6 gti","polo 5 gti","polo 5 r","polo 5 entreprise","polo 4 entreprise","polo 4 gti","polo 3 gti","polo 3 variant","polo 3 flight","polo 3","polo 2 coupe G40","polo 3 entreprise","polo 2 coupe","polo 2","golf 7","golf 8","golf 6","golf r"],
    "nio" : ["et 7","et 9","ec 6","et 5","es 6","ec 7","es 8","es 7"],
    "tazzari" : ["zero"],
    "aptera" : ["solar ev"],
    "suzuki" : ["dzire"],
    "mahindra" : ["kuv 100"],
    "sony" : ["vision s"] 
}

new_rows = []
for brand, models in brands_models_add.items():
    for model in models:
        new_rows.append({'brand': brand, 'model': f'{brand} {model}', 'models_without_brand': model})

# Convert list of new rows to DataFrame
new_rows_df = pd.DataFrame(new_rows)

In [91]:
new_rows_df

,brand,model,models_without_brand
0,toyota,toyota prado,prado
1,toyota,toyota yaris,yaris
2,toyota,toyota corolla,corolla
3,toyota,toyota rav 4,rav 4
4,toyota,toyota hilux,hilux
...,...,...,...
139,tazzari,tazzari zero,zero
140,aptera,aptera solar ev,solar ev
141,suzuki,suzuki dzire,dzire
142,mahindra,mahindra kuv 100,kuv 100


In [92]:
new_rows_df = lowercase_df(new_rows_df)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2884285594.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lower = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


Merging all the  collected data into one dataset

In [93]:
data = pd.concat([scraped_data, model_Brand_Df,new_rows_df], ignore_index=True)

In [94]:
data

,brand,model,models_without_brand
0,abarth,abarth 500,500
1,abarth,abarth 600e,600e
2,abarth,abarth 124,124
3,abarth,abarth grande punto,grande punto
4,abarth,abarth punto evo,punto evo
...,...,...,...
3844,tazzari,tazzari zero,zero
3845,aptera,aptera solar ev,solar ev
3846,suzuki,suzuki dzire,dzire
3847,mahindra,mahindra kuv 100,kuv 100


In [95]:
data = lowercase_df(data)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2884285594.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_lower = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [96]:
#detecting duplicates
data.duplicated(subset= ['model']).sum()

277

In [97]:
data = data.drop_duplicates(subset= ['model'], keep= 'first')

In [98]:
data

,brand,model,models_without_brand
0,abarth,abarth 500,500
1,abarth,abarth 600e,600e
2,abarth,abarth 124,124
3,abarth,abarth grande punto,grande punto
4,abarth,abarth punto evo,punto evo
...,...,...,...
3844,tazzari,tazzari zero,zero
3845,aptera,aptera solar ev,solar ev
3846,suzuki,suzuki dzire,dzire
3847,mahindra,mahindra kuv 100,kuv 100


In [99]:
# Specify the file path where you want to save the CSV file
file_path = 'data.csv'

# Use the to_csv() method to save the DataFrame to a CSV file
data.to_csv(file_path, index=False)

In [100]:
data["models_without_brand"].isna().sum()

0

In [101]:
# Find rows with NaN values in the specified column
data_na_rows = data[data["models_without_brand"].isna()]

# Print the rows with NaN values
print("Rows with NaN values in column '{}':".format("models_without_brand"))
print(data_na_rows)

Rows with NaN values in column 'models_without_brand':
Empty DataFrame
Columns: [brand, model, models_without_brand]
Index: []


In [102]:
'''
# Replace NaN values in 'models_without_brand' with the first word of 'models' column
for model in data["model"]:
    if model.split()[0] == model.split()[1]:
        print(model)
        print(model.split()[0])
        print(model.split()[1])
        data['models_without_brand'].fillna(data['model'].str.split().str[0], inplace=True)
        print(data['model'].str.split().str[0])
'''

'\n# Replace NaN values in \'models_without_brand\' with the first word of \'models\' column\nfor model in data["model"]:\n    if model.split()[0] == model.split()[1]:\n        print(model)\n        print(model.split()[0])\n        print(model.split()[1])\n        data[\'models_without_brand\'].fillna(data[\'model\'].str.split().str[0], inplace=True)\n        print(data[\'model\'].str.split().str[0])\n'

In [103]:
data["models_without_brand"].isna().sum()

0

In [104]:
len(data["brand"].unique())

325

In [105]:
len(data["model"].unique())

3572

In [106]:
len(data["models_without_brand"])

3572

**brand_models_dict: models by brand**

In [107]:
brand_models_dict = data.groupby('brand')['models_without_brand'].apply(lambda x: x.tolist()).to_dict()
brand_models_dict

{'abarth': ['500', '600e', '124', 'grande punto', 'punto evo', 'scorpion'],
 'ac': ['428',
  'cobra',
  'coupe aceca',
  'greyhound',
  'mkvi',
  'roadster ace',
  'shelby cobra'],
 'acrea': ['zest'],
 'acura': ['mdx'],
 'ad tramontana': ['tramontana'],
 'adler': ['diplomat', 'primus', 'trumpf'],
 'aiways': ['u6', 'u5'],
 'aixam': ['city',
  'coupe',
  'crossline',
  'crossover',
  'd-truck',
  'e city',
  'e coupe',
  'e crossover',
  'e-scouty',
  'e-truck',
  'gto',
  'minauto',
  '300',
  '400',
  '500',
  'a540',
  'a721',
  'a741',
  'a751',
  'e-worker',
  'roadline',
  'scouty'],
 'aleko': ['aleko'],
 'alfa romeo': ['33 stradale',
  'giulia',
  'junior',
  'stelvio',
  'tonale',
  '1300',
  '145',
  '146',
  '147',
  '155',
  '156',
  '159',
  '1600',
  '164',
  '166',
  '1750',
  '1900',
  '2000',
  '2600',
  '33',
  '4c',
  '6',
  '75',
  '8c',
  '90',
  'alfasud',
  'alfetta',
  'berlina',
  'brera',
  'duetto',
  'giulietta',
  'gt',
  'gtv',
  'mito',
  'montreal',
  'rz',

testing for empty values in each dict

In [108]:
def has_empty_values(input_dict):
    for value in input_dict.values():
        if not value:
            return True
    return False

In [109]:
# Check if there are empty values in the dictionary
if has_empty_values(brand_models_dict):
    print("The dictionary contains empty values.")
else:
    print("The dictionary does not contain empty values.")

The dictionary does not contain empty values.


In [110]:
import json
def dict_to_json(file_path,dict_name):
    
    # Write dictionary to a JSON file in the project folder
    with open(file_path, 'w') as file:
        json.dump(dict_name, file)

    print(f"File saved to {file_path}") 


In [111]:
brand_models_dict

{'abarth': ['500', '600e', '124', 'grande punto', 'punto evo', 'scorpion'],
 'ac': ['428',
  'cobra',
  'coupe aceca',
  'greyhound',
  'mkvi',
  'roadster ace',
  'shelby cobra'],
 'acrea': ['zest'],
 'acura': ['mdx'],
 'ad tramontana': ['tramontana'],
 'adler': ['diplomat', 'primus', 'trumpf'],
 'aiways': ['u6', 'u5'],
 'aixam': ['city',
  'coupe',
  'crossline',
  'crossover',
  'd-truck',
  'e city',
  'e coupe',
  'e crossover',
  'e-scouty',
  'e-truck',
  'gto',
  'minauto',
  '300',
  '400',
  '500',
  'a540',
  'a721',
  'a741',
  'a751',
  'e-worker',
  'roadline',
  'scouty'],
 'aleko': ['aleko'],
 'alfa romeo': ['33 stradale',
  'giulia',
  'junior',
  'stelvio',
  'tonale',
  '1300',
  '145',
  '146',
  '147',
  '155',
  '156',
  '159',
  '1600',
  '164',
  '166',
  '1750',
  '1900',
  '2000',
  '2600',
  '33',
  '4c',
  '6',
  '75',
  '8c',
  '90',
  'alfasud',
  'alfetta',
  'berlina',
  'brera',
  'duetto',
  'giulietta',
  'gt',
  'gtv',
  'mito',
  'montreal',
  'rz',

In [112]:
# Path to the file (relative or absolute)
file_path = 'brand_models_dict.json'  # Adjust this path if your script is not in the project folder
dict_name = brand_models_dict
dict_to_json(file_path,dict_name)

File saved to brand_models_dict.json


In [113]:
models_list = list(data["model"])
models_list

['abarth 500',
 'abarth 600e',
 'abarth 124',
 'abarth grande punto',
 'abarth punto evo',
 'abarth scorpion',
 'ac 428',
 'ac cobra',
 'ac coupe aceca',
 'ac greyhound',
 'ac mkvi',
 'ac roadster ace',
 'ac shelby cobra',
 'acrea zest',
 'acura mdx',
 'ad tramontana tramontana',
 'adler diplomat',
 'adler primus',
 'adler trumpf',
 'aiways u6',
 'aiways u5',
 'aixam city',
 'aixam coupe',
 'aixam crossline',
 'aixam crossover',
 'aixam d-truck',
 'aixam e city',
 'aixam e coupe',
 'aixam e crossover',
 'aixam e-scouty',
 'aixam e-truck',
 'aixam gto',
 'aixam minauto',
 'aixam 300',
 'aixam 400',
 'aixam 500',
 'aixam a540',
 'aixam a721',
 'aixam a741',
 'aixam a751',
 'aixam e-worker',
 'aixam roadline',
 'aixam scouty',
 'aleko aleko',
 'alfa romeo 33 stradale',
 'alfa romeo giulia',
 'alfa romeo junior',
 'alfa romeo stelvio',
 'alfa romeo tonale',
 'alfa romeo 1300',
 'alfa romeo 145',
 'alfa romeo 146',
 'alfa romeo 147',
 'alfa romeo 155',
 'alfa romeo 156',
 'alfa romeo 159',


In [114]:
def save_list_to_txt(lst, filename, encoding='utf-8'):
    with open(filename, 'w', encoding=encoding) as f:
        for item in lst:
            f.write(str(item) + '\n')

In [115]:
# Save the list to a text file
save_list_to_txt(models_list, 'models_list.txt')

In [116]:
brands_list = [brand for brand in data["brand"].unique() ]

In [117]:
save_list_to_txt(brands_list, 'brands_list.txt')

In [118]:
models_without_brand = [model_no_brand for model_no_brand in data["models_without_brand"]]

In [119]:
models_without_brand

['500',
 '600e',
 '124',
 'grande punto',
 'punto evo',
 'scorpion',
 '428',
 'cobra',
 'coupe aceca',
 'greyhound',
 'mkvi',
 'roadster ace',
 'shelby cobra',
 'zest',
 'mdx',
 'tramontana',
 'diplomat',
 'primus',
 'trumpf',
 'u6',
 'u5',
 'city',
 'coupe',
 'crossline',
 'crossover',
 'd-truck',
 'e city',
 'e coupe',
 'e crossover',
 'e-scouty',
 'e-truck',
 'gto',
 'minauto',
 '300',
 '400',
 '500',
 'a540',
 'a721',
 'a741',
 'a751',
 'e-worker',
 'roadline',
 'scouty',
 'aleko',
 '33 stradale',
 'giulia',
 'junior',
 'stelvio',
 'tonale',
 '1300',
 '145',
 '146',
 '147',
 '155',
 '156',
 '159',
 '1600',
 '164',
 '166',
 '1750',
 '1900',
 '2000',
 '2600',
 '33',
 '4c',
 '6',
 '75',
 '8c',
 '90',
 'alfasud',
 'alfetta',
 'berlina',
 'brera',
 'duetto',
 'giulietta',
 'gt',
 'gtv',
 'mito',
 'montreal',
 'rz',
 'spider',
 'sud',
 'sz es30',
 'gloria concept',
 'kamal',
 'tonale concept',
 'atx',
 'b3',
 'b4',
 'b5',
 'b6',
 'b7',
 'b8',
 'd3 s',
 'd4',
 'd5',
 'xb7',
 'xd3',
 'b10'

In [120]:
save_list_to_txt(models_without_brand, 'models_without_brand_list.txt')

**synonyms dict**

creating two synonym dicts. one for the brands and the other for the models. 
a synonym dict contains possible user inputs.

1) transforming all the brand names into a "standard" format in order to perform the same operations on all of them at the same time : replacing all the special characters with a white space

In [121]:
def replace_special_characters_in_list(words_list):
    # Defining the pattern to match special characters
    pattern = r'[^\w\s]'

    result_list = []

    for word in words_list:
        result = re.sub(pattern, ' ', word)
        result_list.append(result)
    return result_list

In [122]:
std_brands_list = replace_special_characters_in_list(brands_list)

In [123]:
save_list_to_txt(std_brands_list, "std_brands_list.txt", encoding='utf-8')

In [124]:
std_models_list = replace_special_characters_in_list(models_list)

In [125]:
len(std_models_list)

3572

In [126]:
save_list_to_txt(std_models_list, "std_models_list.txt", encoding='utf-8')

2. generating the synonyms' dicts

In [127]:
'''
def classic_synonyms_dict(words_list):
    words_list = replace_special_characters_in_list(words_list)
    synonym_dict = {}
    
        # Creating a list of alternative formats for the model name
    for word in words_list:
        synonyms = [word]  # to Include the original (standardized) name (in the case of one word)
        
        # If the model name contains spaces, create additional synonyms
        if isinstance(word, str):
            if ' ' in word:
                synonyms.append(word.replace(" ", ""))  # Concatenated version
                synonyms.append(word.replace(" ", "-"))  # Hyphenated version
                synonyms.append(word.replace(" ", "_"))  # Hyphenated version
                synonyms.append(word.replace(" ", "&"))
                synonyms.append(word.replace(" ", "$"))
                synonyms.append(word.replace(" ", "%"))
                synonyms.append(word.replace(" ", "#"))
       
        synonym_dict[word] = synonyms

    return synonym_dict
'''    

'\ndef classic_synonyms_dict(words_list):\n    words_list = replace_special_characters_in_list(words_list)\n    synonym_dict = {}\n    \n        # Creating a list of alternative formats for the model name\n    for word in words_list:\n        synonyms = [word]  # to Include the original (standardized) name (in the case of one word)\n        \n        # If the model name contains spaces, create additional synonyms\n        if isinstance(word, str):\n            if \' \' in word:\n                synonyms.append(word.replace(" ", ""))  # Concatenated version\n                synonyms.append(word.replace(" ", "-"))  # Hyphenated version\n                synonyms.append(word.replace(" ", "_"))  # Hyphenated version\n                synonyms.append(word.replace(" ", "&"))\n                synonyms.append(word.replace(" ", "$"))\n                synonyms.append(word.replace(" ", "%"))\n                synonyms.append(word.replace(" ", "#"))\n       \n        synonym_dict[word] = synonyms\n\n

In [128]:
def replace_special_characters(word):
    # Defining the pattern to match special characters
    pattern = r'[^\w\s]'
    # Replace special characters with space
    return re.sub(pattern, ' ', word)

In [129]:
std_data = data.applymap(replace_special_characters)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\66678923.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  std_data = data.applymap(replace_special_characters)


In [130]:
std_data["models_without_brand"].isna().sum()

0

In [131]:
def generate_synonyms(word):
    # Replace special characters in the word first
    word = replace_special_characters(word)
    synonyms = [word]  # include the standardized name
    
    # Creating additional formatted versions if spaces are present
    if ' ' in word:
        synonyms.extend([
            word.replace(" ", ""),   # Concatenated version
            word.replace(" ", "-"),  # Hyphenated version
            word.replace(" ", "_"),  # Underscore version
            word.replace(" ", "&"),
            word.replace(" ", "$"),
            word.replace(" ", "%"),
            word.replace(" ", "#"),
            word.replace(" ", "/"),
        ])
    return synonyms

def process_row(row):
    # Generating synonyms for both 'model' and 'models_without_brand'
    model_synonyms = generate_synonyms(row['model'])
    models_without_brand_synonyms = generate_synonyms(row['models_without_brand'])
    
    # Returning a new series preserving the brand and adding the synonym data
    return pd.Series({
        'brand': row['brand'],
        'model_synonyms': model_synonyms,
        'models_without_brand_synonyms': models_without_brand_synonyms
    })

processed_df = data.apply(process_row, axis=1)


In [132]:
processed_df

,brand,model_synonyms,models_without_brand_synonyms
0,abarth,"[abarth 500, abarth500, abarth-500, abarth_500...",[500]
1,abarth,"[abarth 600e, abarth600e, abarth-600e, abarth_...",[600e]
2,abarth,"[abarth 124, abarth124, abarth-124, abarth_124...",[124]
3,abarth,"[abarth grande punto, abarthgrandepunto, abart...","[grande punto, grandepunto, grande-punto, gran..."
4,abarth,"[abarth punto evo, abarthpuntoevo, abarth-punt...","[punto evo, puntoevo, punto-evo, punto_evo, pu..."
...,...,...,...
3844,tazzari,"[tazzari zero, tazzarizero, tazzari-zero, tazz...",[zero]
3845,aptera,"[aptera solar ev, apterasolarev, aptera-solar-...","[solar ev, solarev, solar-ev, solar_ev, solar&..."
3846,suzuki,"[suzuki dzire, suzukidzire, suzuki-dzire, suzu...",[dzire]
3847,mahindra,"[mahindra kuv 100, mahindrakuv100, mahindra-ku...","[kuv 100, kuv100, kuv-100, kuv_100, kuv&100, k..."


In [133]:
'''
for models in processed_df["models_without_brand_synonyms"]:
    element = models[0]
    elements_to_add = element.split(" ")
    if elements_to_add not in models:
        models.extend(elements_to_add)
        print(models)
    
'''    
        

'\nfor models in processed_df["models_without_brand_synonyms"]:\n    element = models[0]\n    elements_to_add = element.split(" ")\n    if elements_to_add not in models:\n        models.extend(elements_to_add)\n        print(models)\n    \n'

In [134]:
# Specify the file path where you want to save the CSV file
file_path = 'processed_df.csv'

# Use the to_csv() method to save the DataFrame to a CSV file
processed_df.to_csv(file_path, index=False)

In [135]:
def append_dict_values(dict1, dict2):
    """
    Append the values of dict2 to the values of dict1.
    
    Args:
    - dict1 (dict): The first dictionary.
    - dict2 (dict): The second dictionary.
    
    Returns:
    - dict: The updated dictionary with values appended.
    """
    for key in dict1:
        # Check if the key exists in dict2
        if key in dict2 and len(key.split(" ")) >= 3:
            # Append the values of dict2 to the values of dict1 for the corresponding key
            dict1[key].extend(dict2[key])
    return dict1

In [136]:
# Load the DataFrame from the CSV file
df = pd.read_csv(r'C:\Users\ASUS\Desktop\stage_pfe\projet SEO\pfeProject\jupyter_files\processed_df.csv')

models_syn_dict = {}

for index, row in df.iterrows():
    # We need to evaluate these strings to convert them back to lists
    model_synonyms = eval(row['model_synonyms'])
    models_without_brand_synonyms = eval(row['models_without_brand_synonyms'])
    
    # to Use the first element of the model_synonyms list as the key
    key = model_synonyms[0]
    
    # to Combine the rest of the model_synonyms with the models_without_brand_synonyms
    values = model_synonyms[0:] + models_without_brand_synonyms
    
    models_syn_dict[key] = values


print(models_syn_dict)


{'abarth 500': ['abarth 500', 'abarth500', 'abarth-500', 'abarth_500', 'abarth&500', 'abarth$500', 'abarth%500', 'abarth#500', 'abarth/500', '500'], 'abarth 600e': ['abarth 600e', 'abarth600e', 'abarth-600e', 'abarth_600e', 'abarth&600e', 'abarth$600e', 'abarth%600e', 'abarth#600e', 'abarth/600e', '600e'], 'abarth 124': ['abarth 124', 'abarth124', 'abarth-124', 'abarth_124', 'abarth&124', 'abarth$124', 'abarth%124', 'abarth#124', 'abarth/124', '124'], 'abarth grande punto': ['abarth grande punto', 'abarthgrandepunto', 'abarth-grande-punto', 'abarth_grande_punto', 'abarth&grande&punto', 'abarth$grande$punto', 'abarth%grande%punto', 'abarth#grande#punto', 'abarth/grande/punto', 'grande punto', 'grandepunto', 'grande-punto', 'grande_punto', 'grande&punto', 'grande$punto', 'grande%punto', 'grande#punto', 'grande/punto'], 'abarth punto evo': ['abarth punto evo', 'abarthpuntoevo', 'abarth-punto-evo', 'abarth_punto_evo', 'abarth&punto&evo', 'abarth$punto$evo', 'abarth%punto%evo', 'abarth#punt

In [137]:
df

,brand,model_synonyms,models_without_brand_synonyms
0,abarth,"['abarth 500', 'abarth500', 'abarth-500', 'aba...",['500']
1,abarth,"['abarth 600e', 'abarth600e', 'abarth-600e', '...",['600e']
2,abarth,"['abarth 124', 'abarth124', 'abarth-124', 'aba...",['124']
3,abarth,"['abarth grande punto', 'abarthgrandepunto', '...","['grande punto', 'grandepunto', 'grande-punto'..."
4,abarth,"['abarth punto evo', 'abarthpuntoevo', 'abarth...","['punto evo', 'puntoevo', 'punto-evo', 'punto_..."
...,...,...,...
3567,tazzari,"['tazzari zero', 'tazzarizero', 'tazzari-zero'...",['zero']
3568,aptera,"['aptera solar ev', 'apterasolarev', 'aptera-s...","['solar ev', 'solarev', 'solar-ev', 'solar_ev'..."
3569,suzuki,"['suzuki dzire', 'suzukidzire', 'suzuki-dzire'...",['dzire']
3570,mahindra,"['mahindra kuv 100', 'mahindrakuv100', 'mahind...","['kuv 100', 'kuv100', 'kuv-100', 'kuv_100', 'k..."


In [138]:
file_path = 'models_syn_dict.json'
dict_to_json(file_path ,models_syn_dict)

File saved to models_syn_dict.json


In [139]:
# import itertools


# # Function to generate all possible accented combinations of a string
# def generate_accented_combinations(input_str):
#     accent_dict = {
#         "a": ["a", "à", "á", "â", "ã", "ä", "å", "ā", "ă", "ą", "ǎ", "ǟ", "ǡ", "ǻ", "ȁ", "ȃ", "ȧ", "ḁ", "ẚ", "ạ", "ả", "ấ", "ầ", "ẩ", "ẫ", "ậ", "ắ", "ằ", "ẳ", "ẵ", "ặ"],
#         "b": ["b", "ƀ", "ɓ", "ḃ", "ḅ", "ḇ"],
#         "c": ["c", "ç", "ć", "ĉ", "ċ", "č", "ȼ", "ḉ"],
#         "d": ["d", "ď", "đ", "ḋ", "ḍ", "ḏ", "ḓ"],
#         "e": ["e", "è", "é", "ê", "ë", "ē", "ĕ", "ė", "ę", "ě", "ȅ", "ȇ", "ȩ", "ḕ", "ḗ", "ḙ", "ḛ", "ḝ", "ẹ", "ẻ", "ẽ", "ế", "ề", "ể", "ễ", "ệ"],
#         "f": ["f", "ƒ", "ḟ"],
#         "g": ["g", "ĝ", "ğ", "ġ", "ģ", "ǥ", "ǧ", "ǵ", "ḡ"],
#         "h": ["h", "ĥ", "ħ", "ȟ", "ḣ", "ḥ", "ḧ", "ḩ", "ḫ"],
#         "i": ["i", "ì", "í", "î", "ï", "ĩ", "ī", "ĭ", "į", "ı", "ǐ", "ȉ", "ȋ", "ḭ", "ỉ", "ị"],
#         "j": ["j", "ĵ"],
#         "k": ["k", "ķ", "ǩ", "ḱ", "ḳ", "ḵ"],
#         "l": ["l", "ĺ", "ļ", "ľ", "ŀ", "ł", "ḽ", "ḷ", "ḹ", "ḻ", "ḽ"],
#         "m": ["m", "ḿ", "ṁ", "ṃ"],
#         "n": ["n", "ñ", "ń", "ņ", "ň", "ŉ", "ṅ", "ṇ", "ṉ", "ṋ"],
#         "o": ["o", "ò", "ó", "ô", "õ", "ö", "ō", "ŏ", "ő", "ǒ", "ǫ", "ǭ", "ȍ", "ȏ", "ȫ", "ȭ", "ȯ", "ȱ", "ọ", "ỏ", "ố", "ồ", "ổ", "ỗ", "ộ", "ớ", "ờ", "ở", "ỡ", "ợ"],
#         "p": ["p", "ṕ", "ṗ"],
#         "q": ["q", "ɋ"],
#         "r": ["r", "ŕ", "ŗ", "ř", "ȑ", "ȓ", "ṙ", "ṛ", "ṝ", "ṟ"],
#         "s": ["s", "ß", "ś", "ŝ", "ş", "š", "ș", "ṡ", "ṣ", "ṥ", "ṧ", "ṩ"],
#         "t": ["t", "ţ", "ť", "ŧ", "ṫ", "ṭ", "ṯ", "ṱ"],
#         "u": ["u", "ù", "ú", "û", "ü", "ũ", "ū", "ŭ", "ů", "ű", "ų", "ǔ", "ǖ", "ǘ", "ǚ", "ǜ", "ȕ", "ȗ", "ṳ", "ụ", "ủ", "ứ", "ừ", "ử", "ữ", "ự"],
#         "v": ["v", "ṽ", "ṿ"],
#         "w": ["w", "ŵ", "ẁ", "ẃ", "ẅ"],
#         "x": ["x", "ẋ", "ẍ"],
#         "y": ["y", "ý", "ÿ", "ŷ", "ȳ", "ẏ", "ỵ", "ỷ", "ỹ"],
#         "z": ["z", "ź", "ż", "ž", "ẑ", "ẓ", "ẕ"],
        
#     }

#     # Function to replace characters with their accented versions
#     def replace_with_accents(char):
#         return accent_dict.get(char, [char])

#     # Generate all combinations of the string with accented characters
#     combinations = [replace_with_accents(char) for char in input_str]
#     all_combinations = set([''.join(comb) for comb in itertools.product(*combinations)])

#     return all_combinations

# # List of car brand names
# brand_names = std_brands_list

# # Create the synonyms dictionary
# accented_brand_syn_dict = {}
# for brand in brand_names:
#     accented_brand_syn_dict[brand] = generate_accented_combinations(brand)

# # Convert sets to lists for readability
# accented_brand_syn_dict = {key: list(value) for key, value in accented_brand_syn_dict.items()}

# # Print the synonyms dictionary
# for brand, synonyms in accented_brand_syn_dict.items():
#     print(f"{brand}: {synonyms}")

 

In [140]:
brand_synonyms_dict = {brand: generate_synonyms(brand) for brand in df['brand'].unique()}

In [141]:
brand_synonyms_dict

{'abarth': ['abarth'],
 'ac': ['ac'],
 'acrea': ['acrea'],
 'acura': ['acura'],
 'ad tramontana': ['ad tramontana',
  'adtramontana',
  'ad-tramontana',
  'ad_tramontana',
  'ad&tramontana',
  'ad$tramontana',
  'ad%tramontana',
  'ad#tramontana',
  'ad/tramontana'],
 'adler': ['adler'],
 'aiways': ['aiways'],
 'aixam': ['aixam'],
 'aleko': ['aleko'],
 'alfa romeo': ['alfa romeo',
  'alfaromeo',
  'alfa-romeo',
  'alfa_romeo',
  'alfa&romeo',
  'alfa$romeo',
  'alfa%romeo',
  'alfa#romeo',
  'alfa/romeo'],
 'alke': ['alke'],
 'alpina': ['alpina'],
 'alvis': ['alvis'],
 'am general': ['am general',
  'amgeneral',
  'am-general',
  'am_general',
  'am&general',
  'am$general',
  'am%general',
  'am#general',
  'am/general'],
 'amc': ['amc'],
 'amphicar': ['amphicar'],
 'anaig': ['anaig'],
 'apal': ['apal'],
 'ariel': ['ariel'],
 'armstrong siddeley': ['armstrong siddeley',
  'armstrongsiddeley',
  'armstrong-siddeley',
  'armstrong_siddeley',
  'armstrong&siddeley',
  'armstrong$siddeley

In [142]:
#brand_synonyms_dict = append_dict_values(brand_synonyms_dict, accented_brand_syn_dict)

In [143]:
new_synonyms = ["benz","merc","mercedes-benz","mercedes benz","mercedesbenz","mercedes_benz","mercedes&benz","mercedes#benz","mercedesbenz"]
for brand in brand_synonyms_dict.keys():
    if brand.lower() == "mercedes":
        brand_synonyms_dict[brand].extend(new_synonyms)

In [144]:

print(brand_synonyms_dict["mercedes"])

['mercedes', 'benz', 'merc', 'mercedes-benz', 'mercedes benz', 'mercedesbenz', 'mercedes_benz', 'mercedes&benz', 'mercedes#benz', 'mercedesbenz']


In [145]:
new_syn = ["gwm"]
for brand in brand_synonyms_dict.keys():
    if brand.lower() == "great wall":
        brand_synonyms_dict[brand].extend(new_syn)
        print(brand_synonyms_dict[brand])

['great wall', 'greatwall', 'great-wall', 'great_wall', 'great&wall', 'great$wall', 'great%wall', 'great#wall', 'great/wall', 'gwm']


In [146]:
for brand in brand_synonyms_dict.keys():
    if len(brand.split(" ")) >= 2:
        for br in brand.split(" "):
            brand_synonyms_dict[brand].append(br)

In [147]:
models_syn_dict 

{'abarth 500': ['abarth 500',
  'abarth500',
  'abarth-500',
  'abarth_500',
  'abarth&500',
  'abarth$500',
  'abarth%500',
  'abarth#500',
  'abarth/500',
  '500'],
 'abarth 600e': ['abarth 600e',
  'abarth600e',
  'abarth-600e',
  'abarth_600e',
  'abarth&600e',
  'abarth$600e',
  'abarth%600e',
  'abarth#600e',
  'abarth/600e',
  '600e'],
 'abarth 124': ['abarth 124',
  'abarth124',
  'abarth-124',
  'abarth_124',
  'abarth&124',
  'abarth$124',
  'abarth%124',
  'abarth#124',
  'abarth/124',
  '124'],
 'abarth grande punto': ['abarth grande punto',
  'abarthgrandepunto',
  'abarth-grande-punto',
  'abarth_grande_punto',
  'abarth&grande&punto',
  'abarth$grande$punto',
  'abarth%grande%punto',
  'abarth#grande#punto',
  'abarth/grande/punto',
  'grande punto',
  'grandepunto',
  'grande-punto',
  'grande_punto',
  'grande&punto',
  'grande$punto',
  'grande%punto',
  'grande#punto',
  'grande/punto'],
 'abarth punto evo': ['abarth punto evo',
  'abarthpuntoevo',
  'abarth-punto-ev

extending the brands_syn_dict : if the brand name is composed of at least two words , we will add each word in the synonyms dict 

brand_synonyms_dict

In [148]:
brand_synonyms_dict

{'abarth': ['abarth'],
 'ac': ['ac'],
 'acrea': ['acrea'],
 'acura': ['acura'],
 'ad tramontana': ['ad tramontana',
  'adtramontana',
  'ad-tramontana',
  'ad_tramontana',
  'ad&tramontana',
  'ad$tramontana',
  'ad%tramontana',
  'ad#tramontana',
  'ad/tramontana',
  'ad',
  'tramontana'],
 'adler': ['adler'],
 'aiways': ['aiways'],
 'aixam': ['aixam'],
 'aleko': ['aleko'],
 'alfa romeo': ['alfa romeo',
  'alfaromeo',
  'alfa-romeo',
  'alfa_romeo',
  'alfa&romeo',
  'alfa$romeo',
  'alfa%romeo',
  'alfa#romeo',
  'alfa/romeo',
  'alfa',
  'romeo'],
 'alke': ['alke'],
 'alpina': ['alpina'],
 'alvis': ['alvis'],
 'am general': ['am general',
  'amgeneral',
  'am-general',
  'am_general',
  'am&general',
  'am$general',
  'am%general',
  'am#general',
  'am/general',
  'am',
  'general'],
 'amc': ['amc'],
 'amphicar': ['amphicar'],
 'anaig': ['anaig'],
 'apal': ['apal'],
 'ariel': ['ariel'],
 'armstrong siddeley': ['armstrong siddeley',
  'armstrongsiddeley',
  'armstrong-siddeley',
  '

In [149]:
for br in brand_synonyms_dict.keys():
    if len(br.split(" ")) >= 2:
        for word in br.split(" "):
            if word not in brand_synonyms_dict[br]:
                brand_synonyms_dict[br].append(word)


In [150]:
print(brand_synonyms_dict)

{'abarth': ['abarth'], 'ac': ['ac'], 'acrea': ['acrea'], 'acura': ['acura'], 'ad tramontana': ['ad tramontana', 'adtramontana', 'ad-tramontana', 'ad_tramontana', 'ad&tramontana', 'ad$tramontana', 'ad%tramontana', 'ad#tramontana', 'ad/tramontana', 'ad', 'tramontana'], 'adler': ['adler'], 'aiways': ['aiways'], 'aixam': ['aixam'], 'aleko': ['aleko'], 'alfa romeo': ['alfa romeo', 'alfaromeo', 'alfa-romeo', 'alfa_romeo', 'alfa&romeo', 'alfa$romeo', 'alfa%romeo', 'alfa#romeo', 'alfa/romeo', 'alfa', 'romeo'], 'alke': ['alke'], 'alpina': ['alpina'], 'alvis': ['alvis'], 'am general': ['am general', 'amgeneral', 'am-general', 'am_general', 'am&general', 'am$general', 'am%general', 'am#general', 'am/general', 'am', 'general'], 'amc': ['amc'], 'amphicar': ['amphicar'], 'anaig': ['anaig'], 'apal': ['apal'], 'ariel': ['ariel'], 'armstrong siddeley': ['armstrong siddeley', 'armstrongsiddeley', 'armstrong-siddeley', 'armstrong_siddeley', 'armstrong&siddeley', 'armstrong$siddeley', 'armstrong%siddeley'

deleting the "s" from the variations of burby's brand name because the has brand function is not detecting ford when it exists as ford s max (model) correctly : it is returning burby s as brand because it is detecting the "s"

In [151]:
for br in brand_synonyms_dict.keys():
    if br == "burby s":
        for var in brand_synonyms_dict[br]:
            if var == "s":
                brand_synonyms_dict[br].remove(var)
                print(brand_synonyms_dict[br])

   

['burby s', 'burbys', 'burby-s', 'burby_s', 'burby&s', 'burby$s', 'burby%s', 'burby#s', 'burby/s', 'burby']


In [152]:
'''
# Iterate over each model in the models_syn_dict keys
for model in models_syn_dict.keys():
    # Check if the model contains more than one word
    if len(model.split(" ")) >= 2:
        # Iterate over each row in the data DataFrame
        for i, row in data.iterrows():
            # Check if the model is present in the current row and is not a prefix of another model
            if model == row["model"]: #and row["model"][len(model)] == " "):
                # Split the words in the "models_without_brand" column for the current row
                model_words = set(row["models_without_brand"].split(" "))
                # Check if the words in "models_without_brand" are a subset of the words in the model name
                if model_words.issubset(set(model.split(" "))):
                    # Append the words to the dictionary under the corresponding model
                    for word in model_words:
                        if word not in models_syn_dict[model]:
                            models_syn_dict[model].append(word)


'''

'\n# Iterate over each model in the models_syn_dict keys\nfor model in models_syn_dict.keys():\n    # Check if the model contains more than one word\n    if len(model.split(" ")) >= 2:\n        # Iterate over each row in the data DataFrame\n        for i, row in data.iterrows():\n            # Check if the model is present in the current row and is not a prefix of another model\n            if model == row["model"]: #and row["model"][len(model)] == " "):\n                # Split the words in the "models_without_brand" column for the current row\n                model_words = set(row["models_without_brand"].split(" "))\n                # Check if the words in "models_without_brand" are a subset of the words in the model name\n                if model_words.issubset(set(model.split(" "))):\n                    # Append the words to the dictionary under the corresponding model\n                    for word in model_words:\n                        if word not in models_syn_dict[model]:\n  

In [153]:
# Initialize an empty dictionary
models_tokens = {}

# Iterate over each row in the data DataFrame
for index, row in std_data.iterrows():
    # Extract the model name from the "model" column
    model_name = row["model"]
    # Extract the tokens from the "models_without_brand" column
    tokens = row["models_without_brand"].split()
    # Update the models_tokens dictionary with the tokens for the current model
    models_tokens[model_name] = tokens

# Display the models_tokens dictionary
print(models_tokens)
    

{'abarth 500': ['500'], 'abarth 600e': ['600e'], 'abarth 124': ['124'], 'abarth grande punto': ['grande', 'punto'], 'abarth punto evo': ['punto', 'evo'], 'abarth scorpion': ['scorpion'], 'ac 428': ['428'], 'ac cobra': ['cobra'], 'ac coupe aceca': ['coupe', 'aceca'], 'ac greyhound': ['greyhound'], 'ac mkvi': ['mkvi'], 'ac roadster ace': ['roadster', 'ace'], 'ac shelby cobra': ['shelby', 'cobra'], 'acrea zest': ['zest'], 'acura mdx': ['mdx'], 'ad tramontana tramontana': ['tramontana'], 'adler diplomat': ['diplomat'], 'adler primus': ['primus'], 'adler trumpf': ['trumpf'], 'aiways u6': ['u6'], 'aiways u5': ['u5'], 'aixam city': ['city'], 'aixam coupe': ['coupe'], 'aixam crossline': ['crossline'], 'aixam crossover': ['crossover'], 'aixam d truck': ['d', 'truck'], 'aixam e city': ['e', 'city'], 'aixam e coupe': ['e', 'coupe'], 'aixam e crossover': ['e', 'crossover'], 'aixam e scouty': ['e', 'scouty'], 'aixam e truck': ['e', 'truck'], 'aixam gto': ['gto'], 'aixam minauto': ['minauto'], 'aixa

In [154]:
extended_models_syn_dict = append_dict_values(models_syn_dict, models_tokens)

In [155]:
print(extended_models_syn_dict)

{'abarth 500': ['abarth 500', 'abarth500', 'abarth-500', 'abarth_500', 'abarth&500', 'abarth$500', 'abarth%500', 'abarth#500', 'abarth/500', '500'], 'abarth 600e': ['abarth 600e', 'abarth600e', 'abarth-600e', 'abarth_600e', 'abarth&600e', 'abarth$600e', 'abarth%600e', 'abarth#600e', 'abarth/600e', '600e'], 'abarth 124': ['abarth 124', 'abarth124', 'abarth-124', 'abarth_124', 'abarth&124', 'abarth$124', 'abarth%124', 'abarth#124', 'abarth/124', '124'], 'abarth grande punto': ['abarth grande punto', 'abarthgrandepunto', 'abarth-grande-punto', 'abarth_grande_punto', 'abarth&grande&punto', 'abarth$grande$punto', 'abarth%grande%punto', 'abarth#grande#punto', 'abarth/grande/punto', 'grande punto', 'grandepunto', 'grande-punto', 'grande_punto', 'grande&punto', 'grande$punto', 'grande%punto', 'grande#punto', 'grande/punto', 'grande', 'punto'], 'abarth punto evo': ['abarth punto evo', 'abarthpuntoevo', 'abarth-punto-evo', 'abarth_punto_evo', 'abarth&punto&evo', 'abarth$punto$evo', 'abarth%punto

In [156]:
# Path to the file (relative or absolute)
file_path = 'extended_models_syn_dict.json'  # Adjust this path if your script is not in the project folder
dict_name = extended_models_syn_dict
dict_to_json(file_path,dict_name)

File saved to extended_models_syn_dict.json


In [157]:
# Path to the file (relative or absolute)
file_path = 'brand_synonyms_dict.json'  # Adjust this path if your script is not in the project folder
dict_name = brand_synonyms_dict
dict_to_json(file_path,dict_name)

File saved to brand_synonyms_dict.json


In [158]:
def check_exist_stopW(token_dict,stopwords_list):
    stopwords_found = {}
    for key, value in token_dict.items():
        current_stopwords = []
        # Iterating through each token in the value list
        for token in value:
            # Convert the token to lowercase once, then compare with each stopword
            lower_token = token.lower()
            # Check each stopword
            for stop in stopwords_list:
                if lower_token == stop:
                    current_stopwords.append(token)
                    #break  # Stop checking other stopwords once a match is found

        if current_stopwords:
            print(f"Stopwords in '{key}': {current_stopwords}")
            stopwords_found[key] = current_stopwords

    return stopwords_found

In [159]:
models_stopW_dict = check_exist_stopW(extended_models_syn_dict,fr_stops)

Stopwords in 'aixam d truck': ['d']
Stopwords in 'alpina d3 s': ['s']
Stopwords in 'audi e tron s': ['s']
Stopwords in 'audi e tron s sportback': ['s']
Stopwords in 'audi ai trail': ['ai']
Stopwords in 'audi le mans': ['le']
Stopwords in 'audi r8 tdi le mans concept': ['le']
Stopwords in 'bentley t': ['t']
Stopwords in 'bmw m vision next concept': ['m']
Stopwords in 'buick le sabre': ['le']
Stopwords in 'chrysler 200 c': ['c']
Stopwords in 'chrysler 300 c': ['c']
Stopwords in 'chrysler 300 m': ['m']
Stopwords in 'chrysler es': ['es']
Stopwords in 'citroen c crosser': ['c']
Stopwords in 'citroen c elysee': ['c']
Stopwords in 'citroen c zero': ['c']
Stopwords in 'citroen d': ['d']
Stopwords in 'citroen type c': ['c']
Stopwords in 'citroen c aircross concept': ['c']
Stopwords in 'citroen c airdream': ['c']
Stopwords in 'citroen c airlounge': ['c']
Stopwords in 'citroen c airplay': ['c']
Stopwords in 'citroen c buggy': ['c']
Stopwords in 'citroen c cactus': ['c']
Stopwords in 'citroen c cr

In [160]:
models_stopW_dict

{'aixam d truck': ['d'],
 'alpina d3 s': ['s'],
 'audi e tron s': ['s'],
 'audi e tron s sportback': ['s'],
 'audi ai trail': ['ai'],
 'audi le mans': ['le'],
 'audi r8 tdi le mans concept': ['le'],
 'bentley t': ['t'],
 'bmw m vision next concept': ['m'],
 'buick le sabre': ['le'],
 'chrysler 200 c': ['c'],
 'chrysler 300 c': ['c'],
 'chrysler 300 m': ['m'],
 'chrysler es': ['es'],
 'citroen c crosser': ['c'],
 'citroen c elysee': ['c'],
 'citroen c zero': ['c'],
 'citroen d': ['d'],
 'citroen type c': ['c'],
 'citroen c aircross concept': ['c'],
 'citroen c airdream': ['c'],
 'citroen c airlounge': ['c'],
 'citroen c airplay': ['c'],
 'citroen c buggy': ['c'],
 'citroen c cactus': ['c'],
 'citroen c crosser concept': ['c'],
 'citroen c metisse': ['c'],
 'citroen c sportlounge': ['c'],
 'citroen cactus m concept': ['m'],
 'courb c zen': ['c'],
 'daf n torpedo': ['n'],
 'daihatsu d bone': ['d'],
 'daihatsu d compact x over': ['d'],
 'daihatsu xl c': ['c'],
 'fiat 500 l': ['l'],
 'ford 

Stop_words : the unique stopwords that exist in car model names

In [161]:
Stop_words = list()
for stopW_list in models_stopW_dict.values(): 
    for stopW in stopW_list:
        Stop_words.append(stopW)

for m in Stop_words:
    if m.lower() == "de":
        Stop_words.remove(m)

In [162]:
Stop_words

['d',
 's',
 's',
 's',
 'ai',
 'le',
 'le',
 't',
 'm',
 'le',
 'c',
 'c',
 'm',
 'es',
 'c',
 'c',
 'c',
 'd',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'c',
 'm',
 'c',
 'n',
 'd',
 'd',
 'c',
 'l',
 's',
 'm',
 'm',
 'm',
 'm',
 'm',
 'c',
 'c',
 'c',
 't',
 's',
 'm',
 'le',
 'd',
 'm',
 'm',
 'n',
 's',
 'c',
 's',
 'c',
 'c',
 'c',
 'c',
 'c',
 'd',
 'd',
 'd',
 's',
 'd',
 'd',
 'c',
 'c',
 'l',
 'es',
 'c',
 's',
 'sa',
 's',
 's',
 's',
 's',
 'c',
 's',
 't',
 'm',
 'c',
 'm',
 'm',
 'm',
 'm',
 'l',
 'm',
 'le',
 'd',
 'le',
 'c',
 's',
 'la',
 'des',
 's',
 'm',
 'c',
 's',
 's',
 's',
 's',
 'y',
 's',
 'c',
 's',
 'c',
 'c',
 's',
 'd',
 'n',
 's',
 'c',
 'm',
 'm',
 'm',
 's',
 't',
 't',
 'd',
 'd',
 'd',
 't',
 't',
 's',
 'm',
 'c',
 's',
 's',
 'c',
 'eu',
 'se',
 'et',
 'et',
 'et',
 'es',
 'es',
 'es',
 's']

In [163]:
brands_stopW_dict = check_exist_stopW(brand_synonyms_dict,fr_stops)

Stopwords in 'chenard et walcker': ['et']
Stopwords in 'de fremond': ['de']
Stopwords in 'de la chapelle': ['de', 'la']
Stopwords in 'de lorean': ['de']
Stopwords in 'de soto': ['de']
Stopwords in 'de tomaso': ['de']


deleting stopwords from the brands syn dict

In [164]:
stopW_del = []
for stopW_br in brands_stopW_dict.values():
    for mot in stopW_br:
        if len(mot) >=2:
            stopW_del.append(mot)

print(stopW_del)            

['et', 'de', 'de', 'la', 'de', 'de', 'de']


In [165]:
# Create a new dictionary to store the cleaned brand synonyms
cleaned_brand_synonyms_dict = {}

# Iterate over the dictionary
for key, values in brand_synonyms_dict.items():
    # Filter out values that are equal to any of the stopwords
    filtered_values = [value for value in values if value not in stopW_del]
    # Update the values in the original dictionary
    brand_synonyms_dict[key] = filtered_values
    # Update the values in the cleaned dictionary
    cleaned_brand_synonyms_dict[key] = filtered_values

In [166]:
cleaned_brand_synonyms_dict

{'abarth': ['abarth'],
 'ac': ['ac'],
 'acrea': ['acrea'],
 'acura': ['acura'],
 'ad tramontana': ['ad tramontana',
  'adtramontana',
  'ad-tramontana',
  'ad_tramontana',
  'ad&tramontana',
  'ad$tramontana',
  'ad%tramontana',
  'ad#tramontana',
  'ad/tramontana',
  'ad',
  'tramontana'],
 'adler': ['adler'],
 'aiways': ['aiways'],
 'aixam': ['aixam'],
 'aleko': ['aleko'],
 'alfa romeo': ['alfa romeo',
  'alfaromeo',
  'alfa-romeo',
  'alfa_romeo',
  'alfa&romeo',
  'alfa$romeo',
  'alfa%romeo',
  'alfa#romeo',
  'alfa/romeo',
  'alfa',
  'romeo'],
 'alke': ['alke'],
 'alpina': ['alpina'],
 'alvis': ['alvis'],
 'am general': ['am general',
  'amgeneral',
  'am-general',
  'am_general',
  'am&general',
  'am$general',
  'am%general',
  'am#general',
  'am/general',
  'am',
  'general'],
 'amc': ['amc'],
 'amphicar': ['amphicar'],
 'anaig': ['anaig'],
 'apal': ['apal'],
 'ariel': ['ariel'],
 'armstrong siddeley': ['armstrong siddeley',
  'armstrongsiddeley',
  'armstrong-siddeley',
  '

In [167]:
# Path to the file (relative or absolute)
file_path = 'cleaned_brand_synonyms_dict.json'  # Adjust this path if your script is not in the project folder
dict_name = cleaned_brand_synonyms_dict
dict_to_json(file_path,dict_name)

File saved to cleaned_brand_synonyms_dict.json


deleting stopwords from the models syn dict

In [168]:
Stop_words = []
for stopW_list in models_stopW_dict.values(): 
    for stopW in stopW_list:
        if stopW not in ["la", "le", "des","et","de"]:  
            Stop_words.append(stopW)


print(set(Stop_words))      

{'es', 'l', 'y', 'n', 'sa', 'ai', 'eu', 's', 'se', 'd', 'c', 't', 'm'}


In [169]:
# Convert lists to sets
fr_stops_set = set(fr_stops)
Stop_words_set = set(Stop_words)
#stopW_del_set = set(stopW_del)

# Perform set operations
my_stopwords = (fr_stops_set - Stop_words_set) 


In [170]:
my_stopwords

{'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'ce',
 'ces',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'est',
 'et',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'il',
 'ils',
 'j',
 'je',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'ma',
 'mais',
 'me',
 'mes',
 'moi',
 'mon',
 'même',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ont',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 'sera',
 'serai',
 'seraient',
 'serais',
 'serait',
 'seras',
 'serez',
 'seriez',
 'serions',
 'serons',
 'seront',
 'ses'

In [171]:
for stop in my_stopwords:
    #print(stop)
    if stop == "de":
        print("exists")
    
print("none")


exists
none


In [172]:
# Write my_stopwords to a txt file
with open('my_stopwords.txt', 'w') as file:
    for word in my_stopwords:
        file.write(word + '\n')

print("Stopwords have been written to my_stopwords.txt")

Stopwords have been written to my_stopwords.txt


In [173]:

# Create a new dictionary to store the cleaned extended models synonyms
cleaned_models_syn_dict = {}

# Iterate over the dictionary
for key, values in extended_models_syn_dict.items():
    # Filter out values that are equal to any of the stopwords
    filtered_values = [value for value in values if value not in my_stopwords]
    # Update the values in the original dictionary
    #extended_models_syn_dict[key] = filtered_values
    # Update the values in the cleaned dictionary
    cleaned_models_syn_dict[key] = filtered_values



In [174]:
cleaned_models_syn_dict

{'abarth 500': ['abarth 500',
  'abarth500',
  'abarth-500',
  'abarth_500',
  'abarth&500',
  'abarth$500',
  'abarth%500',
  'abarth#500',
  'abarth/500',
  '500'],
 'abarth 600e': ['abarth 600e',
  'abarth600e',
  'abarth-600e',
  'abarth_600e',
  'abarth&600e',
  'abarth$600e',
  'abarth%600e',
  'abarth#600e',
  'abarth/600e',
  '600e'],
 'abarth 124': ['abarth 124',
  'abarth124',
  'abarth-124',
  'abarth_124',
  'abarth&124',
  'abarth$124',
  'abarth%124',
  'abarth#124',
  'abarth/124',
  '124'],
 'abarth grande punto': ['abarth grande punto',
  'abarthgrandepunto',
  'abarth-grande-punto',
  'abarth_grande_punto',
  'abarth&grande&punto',
  'abarth$grande$punto',
  'abarth%grande%punto',
  'abarth#grande#punto',
  'abarth/grande/punto',
  'grande punto',
  'grandepunto',
  'grande-punto',
  'grande_punto',
  'grande&punto',
  'grande$punto',
  'grande%punto',
  'grande#punto',
  'grande/punto',
  'grande',
  'punto'],
 'abarth punto evo': ['abarth punto evo',
  'abarthpuntoe

In [175]:
# Path to the file (relative or absolute)
file_path = 'cleaned_models_syn_dict.json'  # Adjust this path if your script is not in the project folder
dict_name = cleaned_models_syn_dict
dict_to_json(file_path,dict_name)

File saved to cleaned_models_syn_dict.json


handling the wrong labeling of "d une/un/occasion .." because the "d" is being detected as a model

In [176]:
# j = 0
# for token in dataset["Tokenized_Keyword"]:
#     for i in range(len(token)-2):
#         j = i + 1
#         #for j in range(i+1,len(token)-1):
#         if (j<len(token)-1) and (token[i] == "d") and ((token[j] not in cleaned_models_syn_dict) and (token[j] not in cleaned_brand_synonyms_dict) ):
#             token.pop(i)
#             token.pop(j)

In [177]:
dataset["Tokenized_Keyword"]

0                            [accessoires, auto, tunisie]
1                          [accessoire, voiture, tunisie]
2       [maison, volkswagen, tunisie, piece, de, recha...
3                                   [enjoliveur, tunisie]
4                          [accessoire, voiture, tunisie]
                              ...                        
5460                        [location, utilitaire, tunis]
5461                     [circuit, 4x4, tunisie, marmara]
5462                           [excursion, 4x4, hammamet]
5463                             [raid, 4x4, en, tunisie]
5464                               [voyage, tunisie, 4x4]
Name: Tokenized_Keyword, Length: 5465, dtype: object

In [178]:
# def clean_invalid_d_tokens(tokenized_keywords, models_synonyms_dict, brands_synonyms_dict):
#     cleaned_keywords = []

#     for tokens in tokenized_keywords:
        
#         new_tokens = []
#         i = 0
#         while i < len(tokens):
                 
#             if tokens[i] == "d":
#                 if i > 0 and i < len(tokens) - 1:
#                     prev_token = tokens[i - 1]
#                     next_token = tokens[i + 1]
#                     if (prev_token not in models_synonyms_dict and prev_token not in brands_synonyms_dict) and (next_token not in models_synonyms_dict and next_token not in brands_synonyms_dict):
#                         # if next_token not in models_synonyms_dict and next_token not in brands_synonyms_dict:
#                             tokenized_keywords.pop(i)
#                             i += 1  # Skip the "d" token
#                             #continue  # Skip incrementing i again at the end of the loop
#             new_tokens.append(tokens[i])
#             i += 1
#         #cleaned_keywords.append(new_tokens)
#         #print(new_tokens)

#     return new_tokens

# # Apply the function to the entire dataset
# dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(
#     lambda x: clean_invalid_d_tokens(x, cleaned_models_syn_dict, cleaned_brand_synonyms_dict)
# )


deleting not nedded single letters to correct their false labeling as a model when they are not in reality : exp : d une ..

In [179]:
def generer_dic_with_model_single_letter (dic): #dic : dict having the brands as keys and their corresponding models as values
    kl={} #new dict : wanted result: keys : brands ; values: single letter existing in the model
    for key,values in dic.items():
        l=[] #list of brands having a single letters in their corresponding model
        for v in values:
            v = str(v)
            s = v.split()
            for m in s:
                if len(m)==1:
                    l.append(m)
        if l!=[] :           
            kl[key]=l
    return kl

dict_model_singLetter = generer_dic_with_model_single_letter (brand_models_dict)
dict_model_singLetter = {key.lower(): [value.lower() for value in values] for key, values in dict_model_singLetter.items()}


def delete_problematic_single_letter(tokenized_keyword, dict_model_singLetter):
    i = 0
    while i < len(tokenized_keyword):
        token = tokenized_keyword[i]
        if len(token) == 1:
            print("singleLetter_index:", i)
            if i == 0:
                next_token = tokenized_keyword[i + 1] if i + 1 < len(tokenized_keyword) else ""
                print("next_token: ", next_token)
                if next_token not in dict_model_singLetter.keys():
                    tokenized_keyword.pop(i)
                    continue  # skip incrementing i
            elif i == len(tokenized_keyword) - 1:
                previous_token = tokenized_keyword[i - 1] if i - 1 >= 0 else ""
                print("previous_token: ", previous_token)
                if previous_token not in dict_model_singLetter.keys():
                    tokenized_keyword.pop(i)
                    continue  # skip incrementing i
            else:
                next_token = tokenized_keyword[i + 1] if i + 1 < len(tokenized_keyword) else ""
                previous_token = tokenized_keyword[i - 1] if i - 1 >= 0 else ""
                print("next_token: ", next_token)
                print("previous_token: ", previous_token)
                if previous_token not in dict_model_singLetter.keys() and next_token not in dict_model_singLetter.keys():
                    tokenized_keyword.pop(i)
                    continue  # skip incrementing i
        i += 1

    return tokenized_keyword

# Apply the function to the 'Tokenized_Keyword' column of the dataset
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(lambda x: delete_problematic_single_letter(x, dict_model_singLetter))




singleLetter_index: 2
next_token:  tunisie
previous_token:  megane
singleLetter_index: 3
next_token:  tunisie
previous_token:  astra
singleLetter_index: 2
next_token:  tunisie
previous_token:  clio
singleLetter_index: 2
next_token:  tunisie
previous_token:  golf
singleLetter_index: 1
next_token:  208
previous_token:  peugeot
singleLetter_index: 0
next_token:  une
singleLetter_index: 0
next_token:  une
singleLetter_index: 2
next_token:  partir
previous_token:  ttc
singleLetter_index: 4
next_token:  une
previous_token:  recharge
singleLetter_index: 1
next_token:  une
previous_token:  cout
singleLetter_index: 0
next_token:  un
singleLetter_index: 1
previous_token:  renault
singleLetter_index: 1
next_token:  une
previous_token:  cout
singleLetter_index: 4
next_token:  la
previous_token:  voiture
singleLetter_index: 0
next_token:  une
singleLetter_index: 1
next_token:  une
previous_token:  cout
singleLetter_index: 1
next_token:  hydrogene
previous_token:  voiture
singleLetter_index: 0
next_

In [180]:
dict_model_singLetter

{'aixam': ['e', 'e', 'e'],
 'alfa romeo': ['6'],
 'alpina': ['s'],
 'aston martin': ['3'],
 'audi': ['s', 's', 's'],
 'bentley': ['t', 'r', 'r', '6', '9', 'f'],
 'biac': ['5', 'x', 'x', 'f'],
 'bmw': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '8',
  '3',
  '5',
  '4',
  'e',
  'i',
  'm',
  '5',
  '6',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  'i',
  'm',
  'z',
  'i',
  'i',
  'e',
  'e',
  'e',
  'i'],
 'byd': ['u', '3'],
 'chery': ['8', '7', '4', '2'],
 'chrysler': ['c', 'b', 'c', 'm'],
 'citroen': ['x', 'd', 'c', 'h', 'm', '9', 'r'],
 'clenet': ['1', '2'],
 'daf': ['n'],
 'daihatsu': ['3'],
 'darmont': ['v'],
 'dkw': ['f'],
 'ds': ['4', '7', '9', '3', '3', '4', '5', '7', 'x', '3', '4', '7', '9'],
 'excalibur': ['1', '2', '3', '4', '5'],
 'facel vega': ['6'],
 'fiat': ['l', 'x', 'e'],
 'ford': ['m', 'm', 'm', 'm', 'm', 'a', 'b', 't', 'x'],
 'geely': ['c'],
 'great wall': ['5'],
 'honda': ['e', 'r', 'r', '2'],
 'hummer': ['h'],
 'hyundai': ['5', '6', '7', 'h', 

deleting not needed stopwords from the keywords

In [181]:
stopw_del_list = list(my_stopwords)
stopw_del_list.append("de")
stopw_del_set = set(stopw_del_list)

In [182]:
# Iterate over each list in the "Tokenized_Keyword" column and filter out the stopwords
dataset['Tokenized_Keyword'] = dataset['Tokenized_Keyword'].apply(lambda x: [word for word in x if word not in stopw_del_set])

In [183]:
dataset

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,neuve
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, auto, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, rechange]",not mentioned,not mentioned,not mentioned,not mentioned
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
4,accessoire voiture tunisie prix,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",prix,not mentioned,not mentioned,not mentioned
...,...,...,...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned,not mentioned,not mentioned,not mentioned
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned,not mentioned,not mentioned,not mentioned
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned,not mentioned,not mentioned,not mentioned
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned


avoiding the confusion of voiture luxe with the model due first luxe 

In [184]:
for m,v in cleaned_models_syn_dict.items():
    if "luxe" in v:
        br = m.split(" ")
        br.remove("luxe")

 
def replace_luxe_to_luxury(tokens):
    # Iterate over the tokens in the list
    for i, token in enumerate(tokens):
        # Check if "luxe" is found
        if token == 'luxe':
            # Check if the previous token is not in the check_words list
            if i > 0 and tokens[i-1] not in br:
                # Replace "luxe" with "luxury"
                tokens[i] = 'luxury'
    return tokens

# Apply the function to each cell in the "Tokenized_Keyword" column
dataset['Tokenized_Keyword'] = dataset['Tokenized_Keyword'].apply(replace_luxe_to_luxury)


avoiding the confusion of "prix voiture" for exp with the model "pontiac grand prix" 

In [185]:
'''
for m,v in cleaned_models_syn_dict.items():
    if "prix" in v:
        br = m.split(" ")
        br.remove("prix")
        


def replace_prix_to_price(tokens):
    # Iterate over the tokens in the list
    for i, token in enumerate(tokens):
        # Check if "prix" is found
        if token == 'prix':
            # Check if the previous token is not in the check_words list
            if ((i > 0 and i < len(tokens) - 1) and (tokens[i-1] not in br)) or ((i >= 0 and i < len(tokens) ) and tokens[i+1] not in br):
                # Replace "prix" with "price"
                tokens[i] = 'price'
    return tokens

# Apply the function to each cell in the "Tokenized_Keyword" column
dataset['Tokenized_Keyword'] = dataset['Tokenized_Keyword'].apply(replace_prix_to_price)
'''


'\nfor m,v in cleaned_models_syn_dict.items():\n    if "prix" in v:\n        br = m.split(" ")\n        br.remove("prix")\n        \n\n\ndef replace_prix_to_price(tokens):\n    # Iterate over the tokens in the list\n    for i, token in enumerate(tokens):\n        # Check if "prix" is found\n        if token == \'prix\':\n            # Check if the previous token is not in the check_words list\n            if ((i > 0 and i < len(tokens) - 1) and (tokens[i-1] not in br)) or ((i >= 0 and i < len(tokens) ) and tokens[i+1] not in br):\n                # Replace "prix" with "price"\n                tokens[i] = \'price\'\n    return tokens\n\n# Apply the function to each cell in the "Tokenized_Keyword" column\ndataset[\'Tokenized_Keyword\'] = dataset[\'Tokenized_Keyword\'].apply(replace_prix_to_price)\n'

In [186]:
def replace_prix_to_price(tokens):
    # Iterate over the tokens in the list
    for i, token in enumerate(tokens):
        # Check if "prix" is found
        if token == 'prix':
            word_index = i
            if i == 0:
                next_word = tokens[word_index + 1] if word_index + 1 < len(tokens) else None
                prv_word = None
                print("next_word: ", next_word)
                print("prv_word: ", prv_word)
                if next_word not in ["grand", "pontiac"]:
                    tokens[word_index] = 'price'

            elif i == (len(tokens) - 1):
                prv_word = tokens[word_index - 1]
                next_word = None
                print("next_word: ", next_word)
                print("prv_word: ", prv_word)
                if prv_word not in ["grand", "pontiac"]:
                    tokens[word_index] = 'price'
                    
            else:
                next_word = tokens[word_index + 1]
                prv_word = tokens[word_index - 1]
                print("next_word: ", next_word)
                print("prv_word: ", prv_word)
                if prv_word not in ["grand", "pontiac"] and next_word not in ["grand", "pontiac"]:
                    tokens[word_index] = 'price'
                    
    return tokens

In [187]:
#test: 
tokens = ["prix","voiture","kia","tunisie"]
replace_prix_to_price(tokens)

next_word:  voiture
prv_word:  None


['price', 'voiture', 'kia', 'tunisie']

In [188]:
# Apply the function to each cell in the "Tokenized_Keyword" column
dataset['Tokenized_Keyword'] = dataset['Tokenized_Keyword'].apply(replace_prix_to_price)

replacing "a" with "à" to avoid labeling it with "ford a " . 
we will check the neighboring words : if they aren't a model / brand name them we will apply the replacement . 

In [189]:
# def replace_word(words_list,word_to_replace,replacement):
#     for word in words_list:
#         if word == word_to_replace:
#             word_index = words_list.index(word)
#             prv_word = words_list[word_index - 1]
#             next_word = words_list[word_index + 1]
#             if (prv_word not in cleaned_models_syn_dict) or (next_word not in cleaned_models_syn_dict):
#                 words_list[word_index] = replacement
#             if (next_word not in cleaned_brand_synonyms_dict):     
#                 words_list[word_index] = replacement
#     return  words_list           

In [190]:
# def replace_word(words_list, word_to_replace, replacement, cleaned_models_syn_dict, cleaned_brand_synonyms_dict):
#     for i in range(len(words_list)):
#         if words_list[i] == word_to_replace:
#             prv_word = words_list[i - 1] if i > 0 else None
#             next_word = words_list[i + 1] if i < len(words_list) - 1 else None
            
#             replace_condition = False
            
#             if prv_word is not None and prv_word not in cleaned_models_syn_dict:
#                 replace_condition = True
#             if next_word is not None and next_word not in cleaned_models_syn_dict:
#                 replace_condition = True
#             if next_word is not None and next_word not in cleaned_brand_synonyms_dict:
#                 replace_condition = True
            
#             if replace_condition:
#                 words_list[i] = replacement
                
#     return words_list

In [191]:
# #test: 
# words_list = ["mercedes","classe","a"]
# replace_word(words_list, "a", "à", cleaned_models_syn_dict, cleaned_brand_synonyms_dict)


In [192]:
def replace_word(tokens_list, word_to_replace, replacement, cleaned_models_syn_dict, cleaned_brand_synonyms_dict):
    for i,token in enumerate(tokens_list):
        if token == word_to_replace:
            token_idx = i
            if i == 0:
                nxt_word = tokens_list[i+1]
                if (nxt_word not in cleaned_brand_synonyms_dict) and (nxt_word not in cleaned_models_syn_dict):
                    tokens_list[i] = replacement
            elif i == (len(tokens_list) -1) :
                prv_word = tokens_list[i-1]
                if (prv_word not in cleaned_brand_synonyms_dict) and (prv_word not in cleaned_models_syn_dict):
                    tokens_list[i] = replacement
            else:
                nxt_word = tokens_list[i+1]
                prv_word = tokens_list[i-1]     
                if ((prv_word not in cleaned_brand_synonyms_dict) and (prv_word not in cleaned_models_syn_dict)) and ((nxt_word not in cleaned_brand_synonyms_dict) and (nxt_word not in cleaned_models_syn_dict)):
                    tokens_list[i] = replacement
                         
    return tokens_list


In [193]:
# # Test example
tokens_list = ['bmw', 'a',"acheter"]
replace_word(tokens_list, "a", "à", cleaned_models_syn_dict, cleaned_brand_synonyms_dict)

['bmw', 'a', 'acheter']

In [194]:
dataset["Tokenized_Keyword"].apply(lambda x: replace_word(x,"a","à",cleaned_models_syn_dict, cleaned_brand_synonyms_dict))

0                         [accessoires, auto, tunisie]
1                       [accessoire, voiture, tunisie]
2       [maison, volkswagen, tunisie, piece, rechange]
3                                [enjoliveur, tunisie]
4                       [accessoire, voiture, tunisie]
                             ...                      
5460                     [location, utilitaire, tunis]
5461                  [circuit, 4x4, tunisie, marmara]
5462                        [excursion, 4x4, hammamet]
5463                              [raid, 4x4, tunisie]
5464                            [voyage, tunisie, 4x4]
Name: Tokenized_Keyword, Length: 5465, dtype: object

replacing a with à :

In [195]:
vocab_cars = [
    "acheter",
    "vendre",
    "louer",
    "conduire",
    "réparer",
    "entretenir",
    "comparer",
    "chercher",
    "trouver",
    "consulter",
    "assurer",
    "financer",
    "configurer",
    "essayer",
    "importer",
    "exporter",
    "nettoyer",
    "polir",
    "modifier",
    "peindre",
    "enregistrer",
    "réviser",
    "diagnostiquer",
    "inspecter",
    "transporter",
    "remorquer",
    "équiper",
    "améliorer",
    "acheter",
    "garantir",
    "changer",
    "accélérer",
    "freiner",
    "naviguer",
    "installer",
    "payer",
    "sécuriser",
    "analyser",
    "ajuster",
    "apprécier",
    "assister",
    "attacher",
    "chiffrer",
    "contrôler",
    "démonter",
    "dépanner",
    "démarrer",
    "développer",
    "distribuer",
    "doubler",
    "échanger",
    "éliminer",
    "évaluer",
    "expédier",
    "gérer",
    "graver",
    "guider",
    "homologuer",
    "investir",
    "manœuvrer",
    "optimiser",
    "participer",
    "planifier",
    "prévoir",
    "programmer",
    "protéger",
    "publier",
    "réagir",
    "rechercher",
    "recycler",
    "réduire",
    "régler",
    "remplacer",
    "réserver",
    "restaurer",
    "revenir",
    "sélectionner",
    "signaler",
    "souder",
    "stationner",
    "surveiller",
    "tester",
    "utiliser",
    "verrouiller",
    "visiter",
    "vérifier","domicile","moins"
]
def replace_a_with_à(tokens_list,verbes_cars):
    for i , token in enumerate(tokens_list):
        if token == "a" and i< (len(tokens_list) - 2):
            if tokens_list[i+1] in verbes_cars:
                tokens_list[i] = "à"
        else:
            continue
dataset["Tokenized_Keyword"].apply(lambda x: replace_a_with_à(x,vocab_cars))


0       None
1       None
2       None
3       None
4       None
        ... 
5460    None
5461    None
5462    None
5463    None
5464    None
Name: Tokenized_Keyword, Length: 5465, dtype: object

dealing with the wrong labeling of max prix for exp : replacing max with maximum

In [196]:
# for i in range(len(dataset["Tokenized_Keyword"])):
#     dataset["Tokenized_Keyword"][i] = replace_word(dataset["Tokenized_Keyword"][i],"max","maximum")
'''
def replace_max(tokens_list,syn_dict):
    for token in tokens_list:
        if token == "max":
            word_index = tokens_list.index(token)
            prv_word = tokens_list[word_index - 1]
            next_word = tokens_list[word_index + 1]
            if ((word_index >= 1)and (word_index<=len(tokens_list)- 2)) and ((prv_word not in syn_dict) or (next_word not in syn_dict.values())) :
                tokens_list[word_index] = "maximum"
            elif (word_index == 0 ) and (next_word not in syn_dict.values()):    
                tokens_list[word_index] = "maximum"
            elif word_index == (len(tokens_list) - 1) and (prv_word not in syn_dict.values()):
                tokens_list[word_index] = "maximum"


    return tokens_list       

    
# Apply the function to each cell in the "Tokenized_Keyword" column
#dataset['Tokenized_Keyword'] = dataset['Tokenized_Keyword'].apply(replace_max)
'''    

'\ndef replace_max(tokens_list,syn_dict):\n    for token in tokens_list:\n        if token == "max":\n            word_index = tokens_list.index(token)\n            prv_word = tokens_list[word_index - 1]\n            next_word = tokens_list[word_index + 1]\n            if ((word_index >= 1)and (word_index<=len(tokens_list)- 2)) and ((prv_word not in syn_dict) or (next_word not in syn_dict.values())) :\n                tokens_list[word_index] = "maximum"\n            elif (word_index == 0 ) and (next_word not in syn_dict.values()):    \n                tokens_list[word_index] = "maximum"\n            elif word_index == (len(tokens_list) - 1) and (prv_word not in syn_dict.values()):\n                tokens_list[word_index] = "maximum"\n\n\n    return tokens_list       \n\n    \n# Apply the function to each cell in the "Tokenized_Keyword" column\n#dataset[\'Tokenized_Keyword\'] = dataset[\'Tokenized_Keyword\'].apply(replace_max)\n'

In [197]:
'''
def replace_max(tokens_list, syn_dict):
    for i, token in enumerate(tokens_list):
        if token.lower() == "max":
            if i > 0 and is_model_name(tokens_list[i - 1], syn_dict) == False:
                tokens_list[i] = "maximum"
            elif i < len(tokens_list) - 1 and is_model_name(tokens_list[i + 1], syn_dict) == False:
                tokens_list[i] = "maximum"
    return tokens_list

def is_model_name(word, syn_dict):
    for model_name in syn_dict:
        if word.lower() in syn_dict[model_name]:
            return True
    return False
'''


'\ndef replace_max(tokens_list, syn_dict):\n    for i, token in enumerate(tokens_list):\n        if token.lower() == "max":\n            if i > 0 and is_model_name(tokens_list[i - 1], syn_dict) == False:\n                tokens_list[i] = "maximum"\n            elif i < len(tokens_list) - 1 and is_model_name(tokens_list[i + 1], syn_dict) == False:\n                tokens_list[i] = "maximum"\n    return tokens_list\n\ndef is_model_name(word, syn_dict):\n    for model_name in syn_dict:\n        if word.lower() in syn_dict[model_name]:\n            return True\n    return False\n'

In [198]:
def replace_max(tokens_list, syn_dict):
    for i, token in enumerate(tokens_list):
        if token.lower() == "max":
            # Determine if "max" is part of a model name
            is_part_of_model = False
            
            # Check previous word if exists
            if i > 0:
                if is_model_name(tokens_list[i - 1], syn_dict):
                    is_part_of_model = True
            
            # Check next word if exists
            if i < len(tokens_list) - 1:
                if is_model_name(tokens_list[i + 1], syn_dict):
                    is_part_of_model = True
            
            # Replace "max" with "maximum" if it is not part of a model name
            if not is_part_of_model:
                tokens_list[i] = "maximum"
                
    return tokens_list

def is_model_name(word, syn_dict):
    for model_name in syn_dict:
        if word.lower() in syn_dict[model_name]:
            return True
    return False



In [199]:
# test
tokens_list = ["max", "ford", "s","max", "tunisie"]
print(replace_max(tokens_list, cleaned_models_syn_dict))

# Additional test
tokens_list2 = ["max", "accessoires", "ford", "s", "max", "tunisie"]
print(replace_max(tokens_list2, cleaned_models_syn_dict))

['max', 'ford', 's', 'max', 'tunisie']
['maximum', 'accessoires', 'ford', 's', 'max', 'tunisie']


In [200]:
# Applying the function to the "Tokenized_Keyword" column
dataset['Tokenized_Keyword'] = dataset['Tokenized_Keyword'].apply(lambda x: replace_max(x, cleaned_models_syn_dict))

In [201]:
dataset

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,neuve
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, auto, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, rechange]",not mentioned,not mentioned,not mentioned,not mentioned
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned
4,accessoire voiture tunisie prix,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",prix,not mentioned,not mentioned,not mentioned
...,...,...,...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned,not mentioned,not mentioned,not mentioned
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned,not mentioned,not mentioned,not mentioned
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned,not mentioned,not mentioned,not mentioned
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned


creating a function that deletes a specific word from the keyword

In [202]:
def remove_word(tokens_list, word_to_del):
    return [token for token in tokens_list if token != word_to_del]     


In [203]:
# Apply the remove_word function to each list in the Tokenized_Keyword column
word_to_del = "recharge"
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(lambda x: remove_word(x, word_to_del))

In [204]:
# Apply the remove_word function to each list in the Tokenized_Keyword column
word_to_del = "voiturette"
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(lambda x: remove_word(x, word_to_del))

In [205]:
# Apply the remove_word function to each list in the Tokenized_Keyword column
word_to_del = "vitesse"
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(lambda x: remove_word(x, word_to_del))

In [206]:
word_to_del = "rapide"
for row in range(len(dataset)):
    tokenized_keywords = dataset["Tokenized_Keyword"][row]
    for words in tokenized_keywords:
        if words == word_to_del:
            word_to_del_idx = tokenized_keywords.index(word_to_del)
            
            if word_to_del_idx in range(1, len(tokenized_keywords) - 1):
                prv_word = tokenized_keywords[word_to_del_idx - 1]
                nxt_word = tokenized_keywords[word_to_del_idx + 1]
                if (prv_word not in cleaned_brand_synonyms_dict and
                    prv_word not in cleaned_models_syn_dict and
                    nxt_word not in cleaned_brand_synonyms_dict and
                    nxt_word not in cleaned_models_syn_dict):
                    dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)
                    
            elif word_to_del_idx == 0:
                nxt_word = tokenized_keywords[word_to_del_idx + 1]
                if (nxt_word not in cleaned_brand_synonyms_dict and
                    nxt_word not in cleaned_models_syn_dict):
                    dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)
                    
            elif word_to_del_idx == len(tokenized_keywords) - 1:
                prv_word = tokenized_keywords[word_to_del_idx - 1]
                if (prv_word not in cleaned_brand_synonyms_dict and
                    prv_word not in cleaned_models_syn_dict):
                    dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)




C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\14969246.py:27: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\14

finding a solution to "auto" : it is being labeled with "auto union" 

In [207]:
word_to_del = "auto"
for row in range(len(dataset)):
    tokenized_keywords = dataset["Tokenized_Keyword"][row]
    for words in tokenized_keywords:
        if words == word_to_del:
            word_to_del_idx = tokenized_keywords.index(word_to_del)
            # Check if word_to_del_idx is the first element
            if word_to_del_idx == 0:
                if len(tokenized_keywords) > 1:
                    nxt_word = tokenized_keywords[word_to_del_idx + 1]
                    if (nxt_word not in cleaned_brand_synonyms_dict.values() and
                        nxt_word not in cleaned_models_syn_dict.values()):
                        dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)
            # Check if word_to_del_idx is the last element
            elif word_to_del_idx == len(tokenized_keywords) - 1:
                prv_word = tokenized_keywords[word_to_del_idx - 1]
                if (prv_word not in cleaned_brand_synonyms_dict.values() and
                    prv_word not in cleaned_models_syn_dict.values()):
                    dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)
            # Check if word_to_del_idx is in the middle
            else:
                nxt_word = tokenized_keywords[word_to_del_idx + 1]
                prv_word = tokenized_keywords[word_to_del_idx - 1]
                if (prv_word not in cleaned_brand_synonyms_dict.values() and
                    prv_word not in cleaned_models_syn_dict.values() and
                    nxt_word not in cleaned_brand_synonyms_dict.values() and
                    nxt_word not in cleaned_models_syn_dict.values()):
                    dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2567781263.py:28: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset["Tokenized_Keyword"][row] = remove_word(tokenized_keywords, word_to_del)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\

**brand / model labeling**

In [208]:
def has_brand(ds,row,tokens_list, brands_synonyms_dict):
    if "detected brand" not in ds.columns:
        #ds["detected brand"] = 'does not exist'
        res = 'does not exist'
    
    res = 'does not exist'
    inter_max = []
    max_found = 0 
    current_found = 0

    for key in brands_synonyms_dict.keys():
        #print(key)
        inter = set(tokens_list).intersection(brands_synonyms_dict[key])
        if inter:
            current_found = len(inter)
            if current_found > max_found:
                inter_max = list(inter)
                #print("inter max:" , inter_max)
                max_found = current_found
                #print("max found:" , max_found)
                found_brand = key
                #print("found brand:" , found_brand)
    
    if inter_max:
        inter_res = ' '.join(inter_max)
        #print(inter_res)
        #ds.at[row, "detected brand"] = found_brand  # Update the "detected brand" column for the first row
        res = found_brand

    #print(ds.at[row, "detected brand"])
    return res

In [209]:
'''
def has_model(ds,row,tokens_list, models_synonyms_dict,brand_model_dict,brands_synonyms_dict):
    if "detected model" not in ds.columns:
        #ds["detected model"] = 'does not exist'
        res = 'does not exist'
    
    res = 'does not exist'
    inter_max = []
    max_found = 0 
    current_found = 0

    

    for key in models_synonyms_dict.keys():
        
        inter = set(tokens_list).intersection(models_synonyms_dict[key])
        print("inter: " , inter)
        if inter:
            current_found = len(inter)
            print("curr found: ",current_found)
            print("max found:" , max_found)
            if current_found > max_found:
                inter_max = list(inter)
                max_found = current_found
                model = key
                break
    
    if inter_max:
        print("inter_max: ",inter_max)
        inter_res = ' '.join(inter_max)
        #print(inter_res)
        #ds.at[row, "detected model"] = model  # Update the "detected model" column for the first row
        res = inter_res
        print("res:" , res)

    
    if has_brand(ds,row,tokens_list, brands_synonyms_dict) != "not mentionned":
                    print('entered')
                    if res in brand_model_dict[has_brand(ds,row,tokens_list, brands_synonyms_dict)]:
                          model = str(has_brand(ds,row,tokens_list, brands_synonyms_dict)) + " " + str(res)
    
        

    #print(ds.at[row, "detected model"])

    #return res
    if has_brand(ds,row,tokens_list, brands_synonyms_dict) != "not mentionned":
        return model
    else:
        return res
'''

'\ndef has_model(ds,row,tokens_list, models_synonyms_dict,brand_model_dict,brands_synonyms_dict):\n    if "detected model" not in ds.columns:\n        #ds["detected model"] = \'does not exist\'\n        res = \'does not exist\'\n    \n    res = \'does not exist\'\n    inter_max = []\n    max_found = 0 \n    current_found = 0\n\n    \n\n    for key in models_synonyms_dict.keys():\n        \n        inter = set(tokens_list).intersection(models_synonyms_dict[key])\n        print("inter: " , inter)\n        if inter:\n            current_found = len(inter)\n            print("curr found: ",current_found)\n            print("max found:" , max_found)\n            if current_found > max_found:\n                inter_max = list(inter)\n                max_found = current_found\n                model = key\n                break\n    \n    if inter_max:\n        print("inter_max: ",inter_max)\n        inter_res = \' \'.join(inter_max)\n        #print(inter_res)\n        #ds.at[row, "detected mo

In [210]:
# def has_model(ds, row, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict):
#     res = 'does not exist'
#     inter_max = []
#     max_found = 0 
#     current_found = 0

#     for key in models_synonyms_dict.keys():
#         inter = set(tokens_list).intersection(models_synonyms_dict[key])
#         if inter:
#             current_found = len(inter)
#             if current_found > max_found:
#                 inter_max = list(inter)
#                 max_found = current_found
#                 res = key
#                 print("res 1:" , res)

#     out_res = res            

#     detected_brand = has_brand(ds, row, tokens_list, brands_synonyms_dict)
#     if detected_brand != "not mentionned":
#         print("entered")
#         print("detected_br: ",detected_brand)
#         brand_models = brand_model_dict.get(detected_brand, [])
#         print("brand_models:" , brand_models)
#         if res in brand_models:
#             print("entered")
#             final_res = f"{detected_brand} {res}"
#             print("res 2:" ,final_res)

#     out_res = final_res   
    
#     return out_res

In [211]:
#this is the last version that works 99% mashallah 
"""
def has_model(ds, row, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict):
    res = 'does not exist'
    inter_max = []
    max_found = 0 
    current_found = 0

    # Identify the potential model
    for key , values in models_synonyms_dict.items():
        inter = set(tokens_list).intersection(models_synonyms_dict[key])
        #print("inter:" , inter)
        if inter:
            current_found = len(inter)
            if current_found > max_found:
                inter_max = list(inter)
                max_found = current_found
                res = ""
                for w in inter:
                    res += w
                #print("res 1:", res)
                break

    
    final_res = res  # Initialize final_res with res in case the brand does not exist in the keyword

    # Detect the brand
    detected_brand = has_brand(ds, row, tokens_list, brands_synonyms_dict)
    if detected_brand != "does not exist":
        #print("entered")
        #print("detected_brand:", detected_brand)
        brand_models = brand_model_dict.get(detected_brand, [])
        #print("brand_models:", brand_models)
        if res in brand_models:
            #print("entered")
            final_res = str(f"{detected_brand} {res}")
            #print("res 2:", final_res)


    return final_res
"""    


'\ndef has_model(ds, row, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict):\n    res = \'does not exist\'\n    inter_max = []\n    max_found = 0 \n    current_found = 0\n\n    # Identify the potential model\n    for key , values in models_synonyms_dict.items():\n        inter = set(tokens_list).intersection(models_synonyms_dict[key])\n        #print("inter:" , inter)\n        if inter:\n            current_found = len(inter)\n            if current_found > max_found:\n                inter_max = list(inter)\n                max_found = current_found\n                res = ""\n                for w in inter:\n                    res += w\n                #print("res 1:", res)\n                break\n\n    \n    final_res = res  # Initialize final_res with res in case the brand does not exist in the keyword\n\n    # Detect the brand\n    detected_brand = has_brand(ds, row, tokens_list, brands_synonyms_dict)\n    if detected_brand != "does not exist":\n        #p

In [212]:
def has_model(ds, row, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict):
    res = 'does not exist'
    max_found = 0
    model_key = None
    res1 = res  # Initialize res1 with res to avoid UnboundLocalError

    # Convert tokens_list to a set for efficient lookups
    tokens_set = set(tokens_list)
    #print("tokens_set:", tokens_set)

    # Detect the brand first
    detected_brand = has_brand(ds, row, tokens_list, brands_synonyms_dict)
    #print("detected brand:", detected_brand)

    # Identify the potential model
    possible_br = []
    for key, values in models_synonyms_dict.items():
        inter = [value for value in values if all(word in tokens_list for word in value.split())]
        #print(f"Checking model synonyms for {key}: {values}")
        #print("inter 1:", inter)
        if inter:
            #print("inter:", inter)
            current_found = len(list(inter)[0].split())
            if current_found > max_found:
                max_found = current_found
                res = list(inter)[0]
                #print("inter[0]:", res)
                
                if detected_brand != "does not exist":
                    # Using a list comprehension to find keys for the brand_model_dict values matching res
                    possible_br = [key for key, value in brand_model_dict.items() if res in value]
                    #print("possible_br:", possible_br)
                    if detected_brand in possible_br:
                        res1 = f"{detected_brand} {res}"
                        #print("res1:", res1)
                        break
                    else:
                        res1 = res  # Update res1 if detected_brand is not in possible_br
                else:
                    res1 = res

    final_res = res1
    return final_res

In [213]:
data_test = pd.DataFrame()
data_test["Keyword"] = [' achat auto tunisie ']
data_test["Tokenized_Keyword"] = data_test['Keyword'].apply(lambda x: text_to_word_sequence(x,filters= '!"()*+,./:;<=>?@[\]^`{|}~'))
# data_test["Tokenized_Keyword"] = data_test["Tokenized_Keyword"].apply(
#     lambda x: clean_invalid_d_tokens(x, cleaned_models_syn_dict, cleaned_brand_synonyms_dict)
# )
data_test['Tokenized_Keyword'] = data_test['Tokenized_Keyword'].apply(lambda x: [word for word in x if word not in stopw_del_set])
data_test['Tokenized_Keyword'] = data_test['Tokenized_Keyword'].apply(replace_luxe_to_luxury)
data_test['Tokenized_Keyword'] = data_test['Tokenized_Keyword'].apply(replace_prix_to_price)
data_test['Tokenized_Keyword'] = data_test['Tokenized_Keyword'].apply(lambda x: replace_max(x, cleaned_models_syn_dict))
data_test["Tokenized_Keyword"] = data_test["Tokenized_Keyword"].apply(lambda x: remove_word(x, word_to_del = "recharge"))
tokens_list = data_test["Tokenized_Keyword"][0]
#print(tokens_list)

has_model(data_test,0,tokens_list, cleaned_models_syn_dict,brand_models_dict,cleaned_brand_synonyms_dict)


<>:3: SyntaxWarning: invalid escape sequence '\]'
<>:3: SyntaxWarning: invalid escape sequence '\]'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2913166674.py:3: SyntaxWarning: invalid escape sequence '\]'
  data_test["Tokenized_Keyword"] = data_test['Keyword'].apply(lambda x: text_to_word_sequence(x,filters= '!"()*+,./:;<=>?@[\]^`{|}~'))


'does not exist'

In [214]:
has_brand(data_test,0,tokens_list, cleaned_brand_synonyms_dict)

'auto union'

In [215]:

'''
def has_model(ds,row,tokens_list, models_synonyms_dict): #ds: dataset
    if "detected model" not in ds.columns:
        ds["detected model"] = 'does not exist'

    
    for idx, token in enumerate(tokens_list):
        for key in models_synonyms_dict.keys():
            for value in models_synonyms_dict[key]:
                if token == value:
                    if value.isdigit() and 1 <= int(value) <= 9:
                            prev_word = tokens_list[idx - 1] if idx > 0 else None
                            next_word = tokens_list[idx + 1] if idx < len(tokens_list) - 1 else None
                            if prev_word in models_synonyms_dict[key] or next_word in models_synonyms_dict[key]:
                                ds["detected model"][row] = key
                                return True
                            
                    else:
                        ds["detected model"][row] = key
                        return True           
    return False
'''    
    

'\ndef has_model(ds,row,tokens_list, models_synonyms_dict): #ds: dataset\n    if "detected model" not in ds.columns:\n        ds["detected model"] = \'does not exist\'\n\n    \n    for idx, token in enumerate(tokens_list):\n        for key in models_synonyms_dict.keys():\n            for value in models_synonyms_dict[key]:\n                if token == value:\n                    if value.isdigit() and 1 <= int(value) <= 9:\n                            prev_word = tokens_list[idx - 1] if idx > 0 else None\n                            next_word = tokens_list[idx + 1] if idx < len(tokens_list) - 1 else None\n                            if prev_word in models_synonyms_dict[key] or next_word in models_synonyms_dict[key]:\n                                ds["detected model"][row] = key\n                                return True\n                            \n                    else:\n                        ds["detected model"][row] = key\n                        return True           \n 

In [216]:
# # Applying the function to each value in the dictionary
# for brand, models_list in brand_models_dict.items():
#     brand_models_dict[brand] = replace_special_characters_in_list(models_list)


In [217]:
#brand_models_dict

In [218]:
# # Function to tokenize model names using .split()
# def tokenize_model_name(model_name):
#     return model_name.split()

# # Add the tokenization of each model to each list and remove duplicates
# for brand, models_list in brand_models_dict.items():
#     tokenized_models = []
#     for model in models_list:
#         tokenized_models.extend(tokenize_model_name(model))
#     # Remove duplicates by converting to a set, then back to a list
#     unique_tokenized_models = list(set(tokenized_models))
#     brand_models_dict[brand] = unique_tokenized_models

# # Print the updated dictionary
# for brand, models_list in brand_models_dict.items():
#     print(f"{brand}: {models_list}")


In [219]:
# def delete_problematic_single_letter(tokenized_keyword,dict_model_singLetter):
#     for token in tokenized_keyword:
#         if len(token) == 1:
#             singleLetter_index = tokenized_keyword.index(token)
#             print("singleLetter_index:",singleLetter_index)
#             if singleLetter_index == 0:
#                 next_token = tokenized_keyword[singleLetter_index +1]
#                 print("next_token: ",next_token)
#                 if next_token not in dict_model_singLetter.keys():
#                     tokenized_keyword.pop(singleLetter_index)
                    
                    
#             elif singleLetter_index == (len(tokenized_keyword) - 1):  
#                 previous_token = tokenized_keyword[singleLetter_index - 1]
#                 print("previous_token: ",previous_token)
#                 if previous_token not in dict_model_singLetter.keys(): 
#                     tokenized_keyword.pop(singleLetter_index)
#             else:
#                 next_token = tokenized_keyword[singleLetter_index +1]
#                 print("next_token: ",next_token)
#                 previous_token = tokenized_keyword[singleLetter_index - 1]
#                 print("previous_token: ",previous_token) 
#                 if previous_token not in dict_model_singLetter.keys() and next_token not in dict_model_singLetter.keys() : 
#                     tokenized_keyword.pop(singleLetter_index)

#     return tokenized_keyword
                 


            

In [220]:
def generer_dic_with_model_single_letter (dic): #dic : dict having the brands as keys and their corresponding models as values
    kl={} #new dict : wanted result: keys : brands ; values: single letter existing in the model
    for key,values in dic.items():
        l=[] #list of brands having a single letters in their corresponding model
        for v in values:
            v = str(v)
            s = v.split()
            for m in s:
                if len(m)==1:
                    l.append(m)
        if l!=[] :           
            kl[key]=l
    return kl

dict_model_singLetter = generer_dic_with_model_single_letter (brand_models_dict)
dict_model_singLetter = {key.lower(): [value.lower() for value in values] for key, values in dict_model_singLetter.items()}
dict_model_singLetter

def delete_problematic_single_letter(tokenized_keyword, dict_model_singLetter):
    i = 0
    while i < len(tokenized_keyword):
        token = tokenized_keyword[i]
        if len(token) == 1:
            print("singleLetter_index:", i)
            if i == 0:
                next_token = tokenized_keyword[i + 1] if i + 1 < len(tokenized_keyword) else ""
                print("next_token: ", next_token)
                if next_token not in dict_model_singLetter.keys():
                    tokenized_keyword.pop(i)
                    continue  # skip incrementing i
            elif i == len(tokenized_keyword) - 1:
                previous_token = tokenized_keyword[i - 1] if i - 1 >= 0 else ""
                print("previous_token: ", previous_token)
                if previous_token not in dict_model_singLetter.keys():
                    tokenized_keyword.pop(i)
                    continue  # skip incrementing i
            else:
                next_token = tokenized_keyword[i + 1] if i + 1 < len(tokenized_keyword) else ""
                previous_token = tokenized_keyword[i - 1] if i - 1 >= 0 else ""
                print("next_token: ", next_token)
                print("previous_token: ", previous_token)
                if previous_token not in dict_model_singLetter.keys() and next_token not in dict_model_singLetter.keys():
                    tokenized_keyword.pop(i)
                    continue  # skip incrementing i
        i += 1

    return tokenized_keyword

# Apply the function to the 'Tokenized_Keyword' column of the dataset
dataset["Tokenized_Keyword"] = dataset["Tokenized_Keyword"].apply(lambda x: delete_problematic_single_letter(x, dict_model_singLetter))




singleLetter_index: 1
next_token:  208
previous_token:  peugeot
singleLetter_index: 1
previous_token:  renault
singleLetter_index: 1
previous_token:  mg
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 1
previous_token:  renault
singleLetter_index: 1
next_token:  niro
previous_token:  kia
singleLetter_index: 2
next_token:  niro
previous_token:  kia
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 1
next_token:  208
previous_token:  peugeot
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 2
next_token:  niro
previous_token:  kia
singleLetter_index: 1
next_token:  tron
previous_token:  audi
singleLetter_index: 1
next_token:  mehari
previous_token:  citroen
singleLetter_index: 2
next_token:  niro
previous_tok

In [221]:
dict_model_singLetter

{'aixam': ['e', 'e', 'e'],
 'alfa romeo': ['6'],
 'alpina': ['s'],
 'aston martin': ['3'],
 'audi': ['s', 's', 's'],
 'bentley': ['t', 'r', 'r', '6', '9', 'f'],
 'biac': ['5', 'x', 'x', 'f'],
 'bmw': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '8',
  '3',
  '5',
  '4',
  'e',
  'i',
  'm',
  '5',
  '6',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  'i',
  'm',
  'z',
  'i',
  'i',
  'e',
  'e',
  'e',
  'i'],
 'byd': ['u', '3'],
 'chery': ['8', '7', '4', '2'],
 'chrysler': ['c', 'b', 'c', 'm'],
 'citroen': ['x', 'd', 'c', 'h', 'm', '9', 'r'],
 'clenet': ['1', '2'],
 'daf': ['n'],
 'daihatsu': ['3'],
 'darmont': ['v'],
 'dkw': ['f'],
 'ds': ['4', '7', '9', '3', '3', '4', '5', '7', 'x', '3', '4', '7', '9'],
 'excalibur': ['1', '2', '3', '4', '5'],
 'facel vega': ['6'],
 'fiat': ['l', 'x', 'e'],
 'ford': ['m', 'm', 'm', 'm', 'm', 'a', 'b', 't', 'x'],
 'geely': ['c'],
 'great wall': ['5'],
 'honda': ['e', 'r', 'r', '2'],
 'hummer': ['h'],
 'hyundai': ['5', '6', '7', 'h', 

In [222]:
# tokenized_keyword = ["m","prix","voiture","d","occasion","s"]
# delete_problematic_single_letter(tokenized_keyword,dict_model_singLetter)

In [223]:
#def label_keyword_with_singLetter(dataset,tokens_col_idx,brands_synonyms_dict,models_synonyms_dict,dic_marque_lettre):
'''
def label_keyword_with_singLetter(dataset,tokens_col_idx,brands_synonyms_dict,models_synonyms_dict,brand_model_dict):    
    dataset["detected brand"] = ''
    dataset["detected model"] = ''
    dataset["Label"] = ''
    for i in range(len(dataset)):
        label = "unknown"
        #for j in range(len(dataset.loc[i][1])):
        tokens_list = dataset.loc[i][tokens_col_idx]
        
        if (has_model(dataset,i,tokens_list,models_synonyms_dict) != 'does not exist') :#or (label_2(dataset,tokens_list,dic_marque_lettre)=="modele") :
            print("test1")
            label = "modèle"
            dataset["detected model"][i] = has_model(dataset,i,tokens_list,models_synonyms_dict)
            print(dataset["detected model"][i])
            for key,value in brand_model_dict.items():
                print("entered")
                if value == has_model(dataset,i,tokens_list,models_synonyms_dict):
                    #if has_brand(dataset,i,tokens_list, brands_synonyms_dict) != 'does not exist':
                    print("test2")
                    dataset["detected brand"][i] = key
                    break

                else:
                    print("test3")
                    dataset["detected brand"][i] = "not mentionned"

       
        if has_brand(dataset,i,tokens_list, brands_synonyms_dict) != 'does not exist':
            label = "marque"
            print("test4")
            dataset["detected brand"][i] = has_brand(dataset,i,tokens_list, brands_synonyms_dict)
            dataset["detected model"][i] = "not mentionned"
            #print("test1")      

        dataset["Label"][i] = label            
        print(dataset["Label"][i])
'''

'\ndef label_keyword_with_singLetter(dataset,tokens_col_idx,brands_synonyms_dict,models_synonyms_dict,brand_model_dict):    \n    dataset["detected brand"] = \'\'\n    dataset["detected model"] = \'\'\n    dataset["Label"] = \'\'\n    for i in range(len(dataset)):\n        label = "unknown"\n        #for j in range(len(dataset.loc[i][1])):\n        tokens_list = dataset.loc[i][tokens_col_idx]\n        \n        if (has_model(dataset,i,tokens_list,models_synonyms_dict) != \'does not exist\') :#or (label_2(dataset,tokens_list,dic_marque_lettre)=="modele") :\n            print("test1")\n            label = "modèle"\n            dataset["detected model"][i] = has_model(dataset,i,tokens_list,models_synonyms_dict)\n            print(dataset["detected model"][i])\n            for key,value in brand_model_dict.items():\n                print("entered")\n                if value == has_model(dataset,i,tokens_list,models_synonyms_dict):\n                    #if has_brand(dataset,i,tokens_list, b

In [224]:
'''
def label_keyword_with_singLetter(dataset,tokens_col_idx,brands_synonyms_dict,models_synonyms_dict,brand_model_dict):    
    dataset["detected brand"] = ''
    dataset["detected model"] = ''
    dataset["Label"] = ''
    for i in range(len(dataset)):
        label = "unknown"
        #for j in range(len(dataset.loc[i][1])):
        tokens_list = dataset.loc[i][tokens_col_idx]

        if (has_model(dataset,i,tokens_list,models_synonyms_dict) != 'does not exist') :#or (label_2(dataset,tokens_list,dic_marque_lettre)=="modele") :
            print("test1")
            label = "modèle"
            dataset["detected model"][i] = has_model(dataset,i,tokens_list,models_synonyms_dict)
            print(dataset["detected model"][i])
            for key,value in brand_model_dict.items():
                print("entered")
                for values in value:
                    if values == dataset["detected model"][i]:
                        if key == has_brand(dataset,i,tokens_list, brand_model_dict):
                            print("test2")
                            dataset["detected brand"][i] = {key} + ' ' + {values}  
                        break

                    else:
                        print("test3")
                        dataset["detected brand"][i] = "not mentionned"

        elif has_brand(dataset,i,tokens_list, brands_synonyms_dict) != 'does not exist':
            label = "marque"
            print("test4")
            dataset["detected brand"][i] = has_brand(dataset,i,tokens_list, brands_synonyms_dict)
            dataset["detected model"][i] = "not mentionned"
            brand = dataset["detected brand"][i]
            for key,value in brand_model_dict.items():
                if key == brand:
                    for values in value:
                        if values == has_model(dataset,i,tokens_list,brand_model_dict):
                            dataset["detected model"][i] = values
                            label = "modèle"
                        break
                        

    
        dataset["Label"][i] = label            
        print(dataset["Label"][i])

'''        

'\ndef label_keyword_with_singLetter(dataset,tokens_col_idx,brands_synonyms_dict,models_synonyms_dict,brand_model_dict):    \n    dataset["detected brand"] = \'\'\n    dataset["detected model"] = \'\'\n    dataset["Label"] = \'\'\n    for i in range(len(dataset)):\n        label = "unknown"\n        #for j in range(len(dataset.loc[i][1])):\n        tokens_list = dataset.loc[i][tokens_col_idx]\n\n        if (has_model(dataset,i,tokens_list,models_synonyms_dict) != \'does not exist\') :#or (label_2(dataset,tokens_list,dic_marque_lettre)=="modele") :\n            print("test1")\n            label = "modèle"\n            dataset["detected model"][i] = has_model(dataset,i,tokens_list,models_synonyms_dict)\n            print(dataset["detected model"][i])\n            for key,value in brand_model_dict.items():\n                print("entered")\n                for values in value:\n                    if values == dataset["detected model"][i]:\n                        if key == has_brand(data

In [225]:
'''
def label_keyword_with_singLetter(dataset, tokens_col_idx, brands_synonyms_dict, models_synonyms_dict, brand_model_dict):    
    dataset["detected brand"] = ''
    dataset["detected model"] = ''
    dataset["Label"] = ''
    
    for i in range(len(dataset)):
        tokens_list = dataset.loc[i][tokens_col_idx]
        
        detected_brand = has_brand(dataset, i, tokens_list, brands_synonyms_dict)
        detected_model = has_model(dataset, i, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict)

        print("Detected Brand:", detected_brand)
        print("Detected Model:", detected_model)
        
        if detected_brand != 'does not exist' and detected_model != 'does not exist':
            print("test 1")
            dataset.at[i, "Label"] = "modèle"
            dataset.at[i, "detected brand"] = detected_brand
            dataset.at[i, "detected model"] = detected_model
            
        elif detected_brand != 'does not exist':
            print("test 2")
            dataset.at[i, "Label"] = "marque"
            dataset.at[i, "detected brand"] = detected_brand
            dataset.at[i, "detected model"] = "not mentionned"
            
        elif detected_model != 'does not exist':
            print("test 3")
            dataset.at[i, "Label"] = "modèle"
            dataset.at[i, "detected model"] = detected_model
            dataset.at[i, "detected brand"] = "not mentionned"
            
        else:
            print("test 4")
            dataset.at[i, "Label"] = "unknown"
            dataset.at[i, "detected brand"] = "not mentionned"
            dataset.at[i, "detected model"] = "not mentionned"

'''

'\ndef label_keyword_with_singLetter(dataset, tokens_col_idx, brands_synonyms_dict, models_synonyms_dict, brand_model_dict):    \n    dataset["detected brand"] = \'\'\n    dataset["detected model"] = \'\'\n    dataset["Label"] = \'\'\n    \n    for i in range(len(dataset)):\n        tokens_list = dataset.loc[i][tokens_col_idx]\n        \n        detected_brand = has_brand(dataset, i, tokens_list, brands_synonyms_dict)\n        detected_model = has_model(dataset, i, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict)\n\n        print("Detected Brand:", detected_brand)\n        print("Detected Model:", detected_model)\n        \n        if detected_brand != \'does not exist\' and detected_model != \'does not exist\':\n            print("test 1")\n            dataset.at[i, "Label"] = "modèle"\n            dataset.at[i, "detected brand"] = detected_brand\n            dataset.at[i, "detected model"] = detected_model\n            \n        elif detected_brand != \'does

In [226]:

def label_keyword(dataset, tokens_col_idx, brands_synonyms_dict, models_synonyms_dict, brand_model_dict):    
    dataset["detected brand"] = ''
    dataset["detected model"] = ''
    dataset["Label"] = ''
    
    for i in range(len(dataset)):
        tokens_list = dataset.loc[i][tokens_col_idx]
        
        detected_brand = has_brand(dataset, i, tokens_list, brands_synonyms_dict)
        detected_model = has_model(dataset, i, tokens_list, models_synonyms_dict, brand_model_dict, brands_synonyms_dict)

        #print("Detected Brand:", detected_brand)
        #print("Detected Model:", detected_model)
        
        if detected_brand != 'does not exist':
            dataset.at[i, "detected brand"] = detected_brand
        else:
            dataset.at[i, "detected brand"] = "not mentionned"
        
        if detected_model != 'does not exist':
            dataset.at[i, "detected model"] = detected_model
        else:
            dataset.at[i, "detected model"] = "not mentionned"
        
        if detected_brand != 'does not exist' and detected_model != 'does not exist':
            dataset.at[i, "Label"] = "modèle"
            
        elif detected_brand != 'does not exist':
            dataset.at[i, "Label"] = "marque"
            
        elif detected_model != 'does not exist':
            dataset.at[i, "Label"] = "modèle"
            
        else:
            dataset.at[i, "Label"] = "unknown"
        


In [227]:
dataset.columns

Index(['keyword', 'Preprocessed_Keyword', 'Tokenized_Keyword', 'prix',
       'hybride/éléctrique', 'occasion', 'neuve'],
      dtype='object')

In [228]:
dataset["Tokenized_Keyword"]

0                               [accessoires, tunisie]
1                       [accessoire, voiture, tunisie]
2       [maison, volkswagen, tunisie, piece, rechange]
3                                [enjoliveur, tunisie]
4                       [accessoire, voiture, tunisie]
                             ...                      
5460                     [location, utilitaire, tunis]
5461                  [circuit, 4x4, tunisie, marmara]
5462                        [excursion, 4x4, hammamet]
5463                              [raid, 4x4, tunisie]
5464                            [voyage, tunisie, 4x4]
Name: Tokenized_Keyword, Length: 5465, dtype: object

In [229]:
label_keyword(dataset,2,cleaned_brand_synonyms_dict,cleaned_models_syn_dict,brand_models_dict)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2777197491.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tokens_list = dataset.loc[i][tokens_col_idx]


In [230]:
dataset

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,neuve,detected brand,detected model,Label
0,accessoires auto tunisie,accessoires auto tunisie,"[accessoires, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
1,accessoire voiture tunisie,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
2,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"[maison, volkswagen, tunisie, piece, rechange]",not mentioned,not mentioned,not mentioned,not mentioned,volkswagen,not mentionned,marque
3,enjoliveur tunisie,enjoliveur tunisie,"[enjoliveur, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
4,accessoire voiture tunisie prix,accessoire voiture tunisie,"[accessoire, voiture, tunisie]",prix,not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
...,...,...,...,...,...,...,...,...,...,...
5460,location utilitaire tunis,location utilitaire tunis,"[location, utilitaire, tunis]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
5461,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"[circuit, 4x4, tunisie, marmara]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle
5462,excursion 4x4 hammamet,excursion 4x4 hammamet,"[excursion, 4x4, hammamet]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle
5463,raid 4x4 en tunisie,raid 4x4 en tunisie,"[raid, 4x4, tunisie]",not mentioned,not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle


In [231]:
dataset["Label"].value_counts()

Label
modèle     2460
unknown    2382
marque      623
Name: count, dtype: int64

In [232]:
dataset[dataset['Label'] == 'modèle'].to_csv('C:/Users/ASUS/Desktop/stage_pfe/collected data/modele2.csv', index=False)
dataset[dataset['Label'] == 'unknown'].to_csv('C:/Users/ASUS/Desktop/stage_pfe/collected data/unknown2.csv', index=False)
dataset[dataset['Label'] == 'marque'].to_csv('C:/Users/ASUS/Desktop/stage_pfe/collected data/marque2.csv', index=False)

In [233]:
unknown_data = pd.read_csv(r'C:/Users/ASUS/Desktop/stage_pfe/collected data/unknown.csv')
unknown_data  

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,detected brand,detected model,Label
0,accessoire voiture tunisie,accessoire voiture tunisie,"['accessoire', 'voiture', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
1,enjoliveur tunisie,enjoliveur tunisie,"['enjoliveur', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
2,accessoire voiture tunisie prix,accessoire voiture tunisie,"['accessoire', 'voiture', 'tunisie']",prix,not mentioned,not mentioned,not mentionned,not mentionned,unknown
3,accessoire voiture tunisie en ligne,accessoire voiture tunisie en ligne,"['accessoire', 'voiture', 'tunisie', 'ligne']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
4,vente accessoire voiture tunisie,vente accessoire voiture tunisie,"['vente', 'accessoire', 'voiture', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
...,...,...,...,...,...,...,...,...,...
2147,location voiture de luxe mariage tunisie,location voiture de luxe mariage tunisie,"['location', 'voiture', 'luxury', 'mariage', '...",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
2148,location voiture utilitaire en tunisie,location voiture utilitaire en tunisie,"['location', 'voiture', 'utilitaire', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
2149,location voiture prestige tunisie,location voiture prestige tunisie,"['location', 'voiture', 'prestige', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown
2150,vehicule utilitaire tunisie,vehicule utilitaire tunisie,"['vehicule', 'utilitaire', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,not mentionned,unknown


In [234]:
model_data = pd.read_csv(r'C:/Users/ASUS/Desktop/stage_pfe/collected data/modele.csv')
model_data

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,detected brand,detected model,Label
0,prix pièce de rechange kia rio tunisie,piece de rechange kia rio tunisie,"['piece', 'rechange', 'kia', 'rio', 'tunisie']",prix,not mentioned,not mentioned,kia,kia rio,modèle
1,accessoires isuzu dmax tunisie,accessoires isuzu dmax tunisie,"['accessoires', 'isuzu', 'dmax', 'tunisie']",not mentioned,not mentioned,not mentioned,isuzu,dmax,modèle
2,pièce de rechange hyundai i10 tunisie,piece de rechange hyundai i10 tunisie,"['piece', 'rechange', 'hyundai', 'i10', 'tunis...",not mentioned,not mentioned,not mentioned,hyundai,hyundai i10,modèle
3,rétroviseur ford fiesta tunisie prix,retroviseur ford fiesta tunisie,"['retroviseur', 'ford', 'fiesta', 'tunisie']",prix,not mentioned,not mentioned,ford,ford fiesta,modèle
4,accessoire mercedes tunisie,accessoire mercedes tunisie,"['accessoire', 'mercedes', 'tunisie']",not mentioned,not mentioned,not mentioned,mercedes,mercedes,modèle
...,...,...,...,...,...,...,...,...,...
2647,4x4 en tunisie,4x4 en tunisie,"['4x4', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle
2648,circuit 4x4 tunisie marmara,circuit 4x4 tunisie marmara,"['circuit', '4x4', 'tunisie', 'marmara']",not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle
2649,excursion 4x4 hammamet,excursion 4x4 hammamet,"['excursion', '4x4', 'hammamet']",not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle
2650,raid 4x4 en tunisie,raid 4x4 en tunisie,"['raid', '4x4', 'tunisie']",not mentioned,not mentioned,not mentioned,not mentionned,4x4,modèle


In [235]:
model_data.head(60)

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,detected brand,detected model,Label
0,prix pièce de rechange kia rio tunisie,piece de rechange kia rio tunisie,"['piece', 'rechange', 'kia', 'rio', 'tunisie']",prix,not mentioned,not mentioned,kia,kia rio,modèle
1,accessoires isuzu dmax tunisie,accessoires isuzu dmax tunisie,"['accessoires', 'isuzu', 'dmax', 'tunisie']",not mentioned,not mentioned,not mentioned,isuzu,dmax,modèle
2,pièce de rechange hyundai i10 tunisie,piece de rechange hyundai i10 tunisie,"['piece', 'rechange', 'hyundai', 'i10', 'tunis...",not mentioned,not mentioned,not mentioned,hyundai,hyundai i10,modèle
3,rétroviseur ford fiesta tunisie prix,retroviseur ford fiesta tunisie,"['retroviseur', 'ford', 'fiesta', 'tunisie']",prix,not mentioned,not mentioned,ford,ford fiesta,modèle
4,accessoire mercedes tunisie,accessoire mercedes tunisie,"['accessoire', 'mercedes', 'tunisie']",not mentioned,not mentioned,not mentioned,mercedes,mercedes,modèle
5,prix pièce de rechange seat ibiza tunisie,piece de rechange seat ibiza tunisie,"['piece', 'rechange', 'seat', 'ibiza', 'tunisie']",prix,not mentioned,not mentioned,seat,seat ibiza,modèle
6,accessoire toyota hilux tunisie,accessoire toyota hilux tunisie,"['accessoire', 'toyota', 'hilux', 'tunisie']",not mentioned,not mentioned,not mentioned,toyota,toyota hilux,modèle
7,pièce de rechange citroen ax tunisie,piece de rechange citroen ax tunisie,"['piece', 'rechange', 'citroen', 'ax', 'tunisie']",not mentioned,not mentioned,not mentioned,citroen,citroen ax,modèle
8,pièce de rechange hyundai i20 tunisie,piece de rechange hyundai i20 tunisie,"['piece', 'rechange', 'hyundai', 'i20', 'tunis...",not mentioned,not mentioned,not mentioned,hyundai,hyundai i20,modèle
9,prix rétroviseur symbol tunisie,retroviseur symbol tunisie,"['retroviseur', 'symbol', 'tunisie']",prix,not mentioned,not mentioned,not mentionned,symbol,modèle


In [236]:
marque_data = pd.read_csv(r'C:/Users/ASUS/Desktop/stage_pfe/collected data/marque.csv')
marque_data

,keyword,Preprocessed_Keyword,Tokenized_Keyword,prix,hybride/éléctrique,occasion,detected brand,detected model,Label
0,accessoires auto tunisie,accessoires auto tunisie,"['accessoires', 'auto', 'tunisie']",not mentioned,not mentioned,not mentioned,auto union,not mentionned,marque
1,maison volkswagen tunisie pièce de rechange,maison volkswagen tunisie piece de rechange,"['maison', 'volkswagen', 'tunisie', 'piece', '...",not mentioned,not mentioned,not mentioned,volkswagen,not mentionned,marque
2,vente accessoire auto tunisie en gros,vente accessoire auto tunisie en gros,"['vente', 'accessoire', 'auto', 'tunisie', 'gr...",not mentioned,not mentioned,not mentioned,auto union,not mentionned,marque
3,pièce de rechange volkswagen tunisie,piece de rechange volkswagen tunisie,"['piece', 'rechange', 'volkswagen', 'tunisie']",not mentioned,not mentioned,not mentioned,volkswagen,not mentionned,marque
4,pièce de rechange volkswagen sfax,piece de rechange volkswagen sfax,"['piece', 'rechange', 'volkswagen', 'sfax']",not mentioned,not mentioned,not mentioned,volkswagen,not mentionned,marque
...,...,...,...,...,...,...,...,...,...
656,bmw occasion en tunisie,bmw en tunisie,"['bmw', 'tunisie']",not mentioned,not mentioned,occasion,bmw,not mentionned,marque
657,vente voiture bmw occasion tunisie,vente voiture bmw tunisie,"['vente', 'voiture', 'bmw', 'tunisie']",not mentioned,not mentioned,occasion,bmw,not mentionned,marque
658,vente automobile en tunisie,vente automobile en tunisie,"['vente', 'automobile', 'tunisie']",not mentioned,not mentioned,not mentioned,china automobile,not mentionned,marque
659,vente auto tunisie occasion,vente auto tunisie,"['vente', 'auto', 'tunisie']",not mentioned,not mentioned,occasion,auto union,not mentionned,marque


In [237]:
# dataset.to_csv('C:/Users/ASUS/Desktop/stage_pfe/collected data/labeled_data.csv', index=False)

**data augmentation to balance the categories**

In [238]:
import random

def add_random_word(keyword,words_to_add):
    words = keyword.split()
    random_word = random.choice(words_to_add)
    random_position = random.randint(0, len(words))
    words.insert(random_position, random_word)
    return ' '.join(words)

In [239]:
unknown_df = pd.DataFrame()
unknown_df["keyword"] = unknown_data["keyword"] 
unknown_df

,keyword
0,accessoire voiture tunisie
1,enjoliveur tunisie
2,accessoire voiture tunisie prix
3,accessoire voiture tunisie en ligne
4,vente accessoire voiture tunisie
...,...
2147,location voiture de luxe mariage tunisie
2148,location voiture utilitaire en tunisie
2149,location voiture prestige tunisie
2150,vehicule utilitaire tunisie


In [240]:
unknown_df['keyword'] = unknown_df['keyword'].apply(lambda x: add_random_word(x, std_brands_list))

In [241]:
unknown_df

,keyword
0,accessoire nio voiture tunisie
1,enjoliveur london taxi company tunisie
2,accessoire rimac voiture tunisie prix
3,accessoire devinci voiture tunisie en ligne
4,vente accessoire voiture tunisie think
...,...
2147,location voiture de de la chapelle luxe mariag...
2148,location voiture ebro utilitaire en tunisie
2149,location voiture prestige iveco tunisie
2150,vehicule utilitaire tunisie rinspeed


In [242]:
unknown_df["Tokenized_Keyword"] = unknown_df['keyword'].apply(lambda x: text_to_word_sequence(x,filters= '!"()*+,./:;<=>?@[\]^`{|}~'))

<>:1: SyntaxWarning: invalid escape sequence '\]'
<>:1: SyntaxWarning: invalid escape sequence '\]'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\3059819719.py:1: SyntaxWarning: invalid escape sequence '\]'
  unknown_df["Tokenized_Keyword"] = unknown_df['keyword'].apply(lambda x: text_to_word_sequence(x,filters= '!"()*+,./:;<=>?@[\]^`{|}~'))


In [243]:
label_keyword(unknown_df,1,cleaned_brand_synonyms_dict,cleaned_models_syn_dict,brand_models_dict)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4356\2777197491.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tokens_list = dataset.loc[i][tokens_col_idx]


In [244]:
unknown_df

,keyword,Tokenized_Keyword,detected brand,detected model,Label
0,accessoire nio voiture tunisie,"[accessoire, nio, voiture, tunisie]",nio,not mentionned,marque
1,enjoliveur london taxi company tunisie,"[enjoliveur, london, taxi, company, tunisie]",london taxi company,taxi,modèle
2,accessoire rimac voiture tunisie prix,"[accessoire, rimac, voiture, tunisie, prix]",rimac,prix,modèle
3,accessoire devinci voiture tunisie en ligne,"[accessoire, devinci, voiture, tunisie, en, li...",devinci,not mentionned,marque
4,vente accessoire voiture tunisie think,"[vente, accessoire, voiture, tunisie, think]",think,not mentionned,marque
...,...,...,...,...,...
2147,location voiture de de la chapelle luxe mariag...,"[location, voiture, de, de, la, chapelle, luxe...",de la chapelle,luxe,modèle
2148,location voiture ebro utilitaire en tunisie,"[location, voiture, ebro, utilitaire, en, tuni...",ebro,not mentionned,marque
2149,location voiture prestige iveco tunisie,"[location, voiture, prestige, iveco, tunisie]",iveco,not mentionned,marque
2150,vehicule utilitaire tunisie rinspeed,"[vehicule, utilitaire, tunisie, rinspeed]",rinspeed,not mentionned,marque


In [245]:
unknown_df["Label"].value_counts()

Label
modèle    1239
marque     913
Name: count, dtype: int64